<a href="https://colab.research.google.com/github/Rahad31/Kl-FedDis-Research-/blob/main/uniform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch import nn
from typing import Dict

In [ ]:
# Define VAE model
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim, z_dim):
        super(VAE, self).__init__()
        self.x_dim = x_dim
        self.h_dim = h_dim
        self.z_dim = z_dim

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),  # 4x4
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(2048, 256),
            nn.ReLU(),
            nn.Linear(256, 2*z_dim)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 2048),
            nn.ReLU(),
            nn.Unflatten(1, (128, 4, 4)),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, stride=2, padding=1),  # 32x32
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = h[:, :self.z_dim], h[:, self.z_dim:]
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


In [ ]:
# Define VAE training procedure
def vae_train(vae: VAE, trainloader: DataLoader, epochs: int) -> None:
    optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)
    for epoch in range(epochs):
        for i, data in enumerate(trainloader, 0):
            inputs, _ = data
            optimizer.zero_grad()
            recon_x, mu, logvar = vae(inputs)
            loss = vae_loss(recon_x, inputs, mu, logvar)
            loss.backward()
            optimizer.step()


In [ ]:

# Define classification model
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Load CIFAR10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

full_dataset = CIFAR10(root="./data", train=True, download=True, transform=transform)
test_set = CIFAR10(root="./data", train=False, download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:02<00:00, 74167062.11it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# Split dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_set, val_set = torch.utils.data.random_split(full_dataset, [train_size, val_size])


In [ ]:
# Create training and validation loaders
trainloader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
valloader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=2)
testloader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)


In [ ]:
# Define VAE loss function
def vae_loss(recon_x, x, mu, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [ ]:
# Define training procedure for classification model
def train(net: nn.Module, trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    best_acc = 0.0

    for epoch in range(epochs):
        # Training loop
        net.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation loop
        net.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in valloader:
                images, labels = data
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(net.state_dict(), 'best_model.pth')

        print(f"Epoch [{epoch+1}/{epochs}], Training Loss: {running_loss / (i+1):.3f}, Validation Accuracy: {val_acc:.2f}%")

In [ ]:
# Define evaluation procedure
def evaluate(net: nn.Module, testloader: DataLoader) -> float:
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [ ]:
def initialize_clients(trainset, transform, batch_size, num_clients):
    clients = {}
    for i in range(num_clients):
        client_trainset = torch.utils.data.Subset(trainset, range(i * len(trainset) // num_clients, (i + 1) * len(trainset) // num_clients))
        client_trainloader = torch.utils.data.DataLoader(client_trainset, batch_size=batch_size, shuffle=True)
        clients[f"client_{i}"] = client_trainloader
    return clients

In [ ]:
def get_distribution_info(vae: VAE) -> Dict:
    # Implement the logic to extract distribution information from the VAE
    # This can involve computing statistics, parameters, or any other relevant information
    # that can be used to generate augmented data

    # Example implementation:
    distribution_info = {
        "mean": vae.encoder[-1].bias.data.cpu().numpy(),
        "std": torch.exp(0.5 * vae.encoder[-1].weight.data).cpu().numpy()
    }

    return distribution_info

In [ ]:
def send_distribution_info(distribution_info: Dict) -> None:
    # Implement the logic to send the distribution information to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the information

    # Example implementation:
    # Send the distribution information to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass


In [ ]:
def generate_augmented_data(vae: VAE, distribution_info: Dict) -> torch.Tensor:
    # Generate augmented data using Uniform distribution
    mean = distribution_info["mean"].mean().item()  # Convert numpy array to float
    std = distribution_info["std"].mean().item()  # Convert numpy array to float
    augmented_data = torch.FloatTensor(64, vae.z_dim).uniform_(mean - std, mean + std)
    return augmented_data


In [ ]:
def federated_train(net: nn.Module, vae: VAE, trainloaders: Dict[str, DataLoader], trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    for epoch in range(epochs):
        for client_id, client_trainloader in trainloaders.items():
            # Train VAE on client data
            vae_train(vae, client_trainloader, epochs=10)

            # Share distribution information with global server
            distribution_info = get_distribution_info(vae)
            send_distribution_info(distribution_info)

            # Receive distribution information from other clients
            other_distribution_info = receive_distribution_info()

            # Generate augmented data using received distribution information
            augmented_data = generate_augmented_data(vae, other_distribution_info)

            # Train classification model using local, augmented, and validation data
            train(net, client_trainloader, valloader, epochs=10)

            # Send model updates to global server
            send_model_update(client_id, net.state_dict())

In [ ]:
# Define logic to receive distribution information from global server
def receive_distribution_info() -> Dict:
    # Receive distribution information logic
    distribution_info = {
        "mean": np.zeros(20),  # Adjust the size based on your latent space dimension
        "std": np.ones(20)
    }
    return distribution_info

In [ ]:
def send_model_update(client_id: str, model_update: Dict) -> None:
    # Implement the logic to send the model update to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the model update

    # Example implementation:
    # Send the model update to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass

In [ ]:
# Define global server procedure
def global_server() -> None:
    net = Net()
    x_dim = 3 * 32 * 32  # CIFAR-10 input size
    h_dim = 400
    z_dim = 20
    vae = VAE(x_dim, h_dim, z_dim)  # Initialize VAE object with required arguments

    # Initialize clients
    num_clients = 5  # Define the number of clients
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=num_clients)

    # Train model using FedDIS
    federated_train(net, vae, clients, trainloader, valloader, epochs=10)

    # Evaluate final model
    test_accuracy = evaluate(net, testloader)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

if __name__ == "__main__":
    global_server()


Epoch [1/10], Training Loss: 2.303, Validation Accuracy: 12.83%
Epoch [2/10], Training Loss: 2.301, Validation Accuracy: 13.90%
Epoch [3/10], Training Loss: 2.299, Validation Accuracy: 13.75%
Epoch [4/10], Training Loss: 2.296, Validation Accuracy: 13.71%
Epoch [5/10], Training Loss: 2.294, Validation Accuracy: 13.32%
Epoch [6/10], Training Loss: 2.290, Validation Accuracy: 13.18%
Epoch [7/10], Training Loss: 2.286, Validation Accuracy: 13.15%
Epoch [8/10], Training Loss: 2.280, Validation Accuracy: 14.79%
Epoch [9/10], Training Loss: 2.273, Validation Accuracy: 17.14%
Epoch [10/10], Training Loss: 2.263, Validation Accuracy: 18.26%
Epoch [1/10], Training Loss: 2.255, Validation Accuracy: 20.06%
Epoch [2/10], Training Loss: 2.239, Validation Accuracy: 21.04%
Epoch [3/10], Training Loss: 2.218, Validation Accuracy: 21.98%
Epoch [4/10], Training Loss: 2.189, Validation Accuracy: 23.01%
Epoch [5/10], Training Loss: 2.158, Validation Accuracy: 23.58%
Epoch [6/10], Training Loss: 2.129, Val

In [ ]:
from scipy.stats import truncnorm
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from typing import Dict, Tuple

# Define VAE model
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim, z_dim):
        super(VAE, self).__init__()
        self.x_dim = x_dim
        self.h_dim = h_dim
        self.z_dim = z_dim

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),  # 4x4
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(2048, 256),
            nn.ReLU(),
            nn.Linear(256, 2*z_dim)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 2048),
            nn.ReLU(),
            nn.Unflatten(1, (128, 4, 4)),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, stride=2, padding=1),  # 32x32
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = h[:, :self.z_dim], h[:, self.z_dim:]
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


# Define VAE training procedure
def vae_train(vae: VAE, trainloader: DataLoader, epochs: int) -> None:
    optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    for epoch in range(epochs):
        vae.train()
        for i, data in enumerate(trainloader, 0):
            inputs, _ = data
            optimizer.zero_grad()
            recon_x, mu, logvar = vae(inputs)
            loss = vae_loss(recon_x, inputs, mu, logvar)
            loss.backward()
            optimizer.step()

        scheduler.step()


# Define classification model
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Load CIFAR10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

full_dataset = CIFAR10(root="./data", train=True, download=True, transform=transform)
test_set = CIFAR10(root="./data", train=False, download=True, transform=transform)

# Split dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_set, val_set = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Create training and validation loaders
trainloader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
valloader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=2)
testloader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)



# Define VAE loss function
def vae_loss(recon_x, x, mu, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Define training procedure for classification model
def train(net: nn.Module, trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    best_acc = 0.0

    for epoch in range(epochs):
        # Training loop
        net.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        scheduler.step()

        # Validation loop
        net.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in valloader:
                images, labels = data
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(net.state_dict(), 'best_model.pth')

        print(f"Epoch [{epoch+1}/{epochs}], Training Loss: {running_loss / (i+1):.3f}, Validation Accuracy: {val_acc:.2f}%")




# Define evaluation procedure
def evaluate(net: nn.Module, testloader: DataLoader) -> float:
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy




# Initialize clients
def initialize_clients(trainset, transform, batch_size, num_clients):
    clients = {}
    for i in range(num_clients):
        client_trainset = torch.utils.data.Subset(trainset, range(i * len(trainset) // num_clients, (i + 1) * len(trainset) // num_clients))
        client_trainloader = torch.utils.data.DataLoader(client_trainset, batch_size=batch_size, shuffle=True)
        clients[f"client_{i}"] = client_trainloader
    return clients

def get_distribution_info(vae: VAE) -> Dict:
    # Implement the logic to extract distribution information from the VAE
    # This can involve computing statistics, parameters, or any other relevant information
    # that can be used to generate augmented data

    # Example implementation:
    distribution_info = {
        "mean": vae.encoder[-1].bias.data.cpu().numpy(),
        "std": torch.exp(0.5 * vae.encoder[-1].weight.data).cpu().numpy()
    }

    return distribution_info

def send_distribution_info(distribution_info: Dict) -> None:
    # Implement the logic to send the distribution information to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the information

    # Example implementation:
    # Send the distribution information to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass


# Define logic to generate augmented data using Truncated Normal distribution
def generate_augmented_data(vae: VAE, distribution_info: Dict) -> torch.Tensor:
    # Generate augmented data using Uniform distribution
    mean = distribution_info["mean"].mean().item()  # Convert numpy array to float
    std = distribution_info["std"].mean().item()  # Convert numpy array to float
    augmented_data = torch.FloatTensor(64, vae.z_dim).uniform_(mean - std, mean + std)
    return augmented_data





def federated_train(net: nn.Module, vae: VAE, trainloaders: Dict[str, DataLoader], trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    for epoch in range(epochs):
        for client_id, client_trainloader in trainloaders.items():
            # Train VAE on client data
            vae_train(vae, client_trainloader, epochs=10)

            # Share distribution information with global server
            distribution_info = get_distribution_info(vae)
            send_distribution_info(distribution_info)

            # Receive distribution information from other clients
            other_distribution_info = receive_distribution_info()

            # Generate augmented data using received distribution information
            augmented_data = generate_augmented_data(vae, other_distribution_info)

            # Train classification model using local, augmented, and validation data
            train(net, client_trainloader, valloader, epochs=10)

            # Send model updates to global server
            send_model_update(client_id, net.state_dict())


def receive_distribution_info() -> Dict:
    # Implement the logic to receive the distribution information from the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to receive the information

    # Example implementation:
    # Receive the distribution information from the global server using a network protocol
    # For example, you can use the `socket` module to receive the information over a network
    # or use a message queue like `RabbitMQ` to receive the information
    distribution_info = {
        "mean": np.zeros(20),
        "std": np.ones(20)
    }
    return distribution_info

def send_model_update(client_id: str, model_update: Dict) -> None:
    # Implement the logic to send the model update to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the model update

    # Example implementation:
    # Send the model update to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass

# Define global server procedure
# Define global server procedure
def global_server() -> None:
    net = Net()
    x_dim = 3 * 32 * 32  # CIFAR-10 input size
    h_dim = 400
    z_dim = 20
    vae = VAE(x_dim, h_dim, z_dim)  # Initialize VAE object with required arguments

    # Initialize clients
    num_clients = 5  # Define the number of clients
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=num_clients)

    # Train model using FedDIS
    federated_train(net, vae, clients, trainloader, valloader, epochs=10)

    # Evaluate final model
    test_accuracy = evaluate(net, testloader)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

if __name__ == "__main__":
    global_server()


100%|██████████| 170498071/170498071 [00:03<00:00, 48402826.90it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch [1/10], Training Loss: 2.304, Validation Accuracy: 10.06%
Epoch [2/10], Training Loss: 2.303, Validation Accuracy: 10.20%
Epoch [3/10], Training Loss: 2.302, Validation Accuracy: 10.31%
Epoch [4/10], Training Loss: 2.301, Validation Accuracy: 10.48%
Epoch [5/10], Training Loss: 2.301, Validation Accuracy: 10.32%
Epoch [6/10], Training Loss: 2.300, Validation Accuracy: 10.08%
Epoch [7/10], Training Loss: 2.299, Validation Accuracy: 10.08%
Epoch [8/10], Training Loss: 2.298, Validation Accuracy: 10.08%
Epoch [9/10], Training Loss: 2.296, Validation Accuracy: 10.61%
Epoch [10/10], Training Loss: 2.295, Validation Accuracy: 11.79%
Epoch [1/10], Training Loss: 2.294, Validation Accuracy: 13.75%
Epoch [2/10], Training Loss: 2.291, Validation Accuracy: 16.63%
Epoch [3/10], Training Loss: 2.287, Validation Accuracy: 18.12%
Epoch [4/10], Training Loss: 2.280, Validation Accuracy: 18.44%
Epoch [5/10], 

In [ ]:
 import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from typing import Dict, Tuple
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# Define VAE model
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim, z_dim):
        super(VAE, self).__init__()
        self.x_dim = x_dim
        self.h_dim = h_dim
        self.z_dim = z_dim

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),  # 4x4
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(2048, 256),
            nn.ReLU(),
            nn.Linear(256, 2*z_dim)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 2048),
            nn.ReLU(),
            nn.Unflatten(1, (128, 4, 4)),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, stride=2, padding=1),  # 32x32
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = h[:, :self.z_dim], h[:, self.z_dim:]
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


# Define VAE training procedure
def vae_train(vae: VAE, trainloader: DataLoader, epochs: int) -> None:
    optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    for epoch in range(epochs):
        vae.train()
        for i, data in enumerate(trainloader, 0):
            inputs, _ = data
            optimizer.zero_grad()
            recon_x, mu, logvar = vae(inputs)
            loss = vae_loss(recon_x, inputs, mu, logvar)
            loss.backward()
            optimizer.step()

        scheduler.step()


# Define classification model
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Load CIFAR10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

full_dataset = CIFAR10(root="./data", train=True, download=True, transform=transform)
test_set = CIFAR10(root="./data", train=False, download=True, transform=transform)

# Split dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_set, val_set = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Create training and validation loaders
trainloader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
valloader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=2)
testloader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)



# Define VAE loss function
def vae_loss(recon_x, x, mu, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Define training procedure for classification model
def train(net: nn.Module, trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    best_acc = 0.0

    for epoch in range(epochs):
        # Training loop
        net.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        scheduler.step()

        # Validation loop
        net.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in valloader:
                images, labels = data
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(net.state_dict(), 'best_model.pth')

        print(f"Epoch [{epoch+1}/{epochs}], Training Loss: {running_loss / (i+1):.3f}, Validation Accuracy: {val_acc:.2f}%")

# Define evaluation procedure
def evaluate(net: nn.Module, testloader: DataLoader) -> Tuple[float, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    net.eval()
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total

    # Calculate confusion matrix
    cm = confusion_matrix(all_labels, all_predictions)
    print(f"Confusion Matrix:\n{cm}")

    # Calculate TP, FP, TN, FN for each class
    tp = np.diag(cm)
    fp = cm.sum(axis=0) - tp
    fn = cm.sum(axis=1) - tp
    tn = cm.sum() - (fp + fn + tp)

    # Calculate precision, recall, and F1 score for each class
    precision = precision_score(all_labels, all_predictions, average=None)
    recall = recall_score(all_labels, all_predictions, average=None)
    f1 = f1_score(all_labels, all_predictions, average=None)

    return accuracy, tp, fp, tn, fn, precision, recall, f1

# Initialize clients
def initialize_clients(trainset, transform, batch_size, num_clients):
    clients = {}
    for i in range(num_clients):
        client_trainset = torch.utils.data.Subset(trainset, range(i * len(trainset) // num_clients, (i + 1) * len(trainset) // num_clients))
        client_trainloader = torch.utils.data.DataLoader(client_trainset, batch_size=batch_size, shuffle=True)
        clients[f"client_{i}"] = client_trainloader
    return clients

def get_distribution_info(vae: VAE) -> Dict:
    # Implement the logic to get the distribution information from the VAE model
    # This can involve extracting the mean and standard deviation of the latent space
    # and sending this information to the global server for use in generating augmented data

    # Example implementation:
    distribution_info = {
        "mean": vae.encoder[-1].bias.data.cpu().numpy(),
        "std": torch.exp(0.5 * vae.encoder[-1].weight.data).cpu().numpy()
    }

    return distribution_info

def send_distribution_info(distribution_info: Dict) -> None:
    # Implement the logic to send the distribution information to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the information

    # Example implementation:
    # Send the distribution information to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass

# Define logic to generate augmented data using Truncated Normal distribution
def generate_augmented_data(vae: VAE, distribution_info: Dict) -> torch.Tensor:
    # Generate augmented data using Uniform distribution
    mean = distribution_info["mean"].mean().item()  # Convert numpy array to float
    std = distribution_info["std"].mean().item()  # Convert numpy array to float
    augmented_data = torch.FloatTensor(64, vae.z_dim).uniform_(mean - std, mean + std)
    return augmented_data

def federated_train(net: nn.Module, vae: VAE, trainloaders: Dict[str, DataLoader], trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    for epoch in range(epochs):
        for client_id, client_trainloader in trainloaders.items():
            # Train VAE on client data
            vae_train(vae, client_trainloader, epochs=10)

            # Share distribution information with global server
            distribution_info = get_distribution_info(vae)
            send_distribution_info(distribution_info)

            # Receive distribution information from other clients
            other_distribution_info = receive_distribution_info()

            # Generate augmented data using received distribution information
            augmented_data = generate_augmented_data(vae, other_distribution_info)

            # Train classification model using local, augmented, and validation data
            train(net, client_trainloader, valloader, epochs=10)

            # Send model updates to global server
            send_model_update(client_id, net.state_dict())

def receive_distribution_info() -> Dict:
    # Implement the logic to receive the distribution information from the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to receive the information

    # Example implementation:
    # Receive the distribution information from the global server using a network protocol
    # For example, you can use the `socket` module to receive the information over a network
    # or use a message queue like `RabbitMQ` to receive the information
    distribution_info = {
        "mean": np.zeros(20),
        "std": np.ones(20)
    }
    return distribution_info

def send_model_update(client_id: str, model_update: Dict) -> None:
    # Implement the logic to send the model update to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the model update

    # Example implementation:
    # Send the model update to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass

# Define global server procedure
def global_server() -> None:
    net = Net()
    x_dim = 3 * 32 * 32  # CIFAR-10 input size
    h_dim = 400
    z_dim = 20
    vae = VAE(x_dim, h_dim, z_dim)  # Initialize VAE object with required arguments

    # Initialize clients
    num_clients = 5  # Define the number of clients
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=num_clients)

    # Train model using FedDIS
    federated_train(net, vae, clients, trainloader, valloader, epochs=10)

    # Evaluate final model
    test_accuracy, tp, fp, tn, fn, precision, recall, f1 = evaluate(net, testloader)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

    print("True Positives (TP):", tp)
    print("False Positives (FP):", fp)
    print("True Negatives (TN):", tn)
    print("False Negatives (FN):", fn)
    print(":", fn+tn+tp+fp)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

if __name__ == "__main__":
    global_server()


100%|██████████| 170498071/170498071 [00:10<00:00, 16168347.18it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch [1/10], Training Loss: 2.305, Validation Accuracy: 9.72%
Epoch [2/10], Training Loss: 2.304, Validation Accuracy: 9.72%
Epoch [3/10], Training Loss: 2.303, Validation Accuracy: 9.72%
Epoch [4/10], Training Loss: 2.302, Validation Accuracy: 9.73%
Epoch [5/10], Training Loss: 2.301, Validation Accuracy: 9.86%
Epoch [6/10], Training Loss: 2.301, Validation Accuracy: 10.91%
Epoch [7/10], Training Loss: 2.300, Validation Accuracy: 12.53%
Epoch [8/10], Training Loss: 2.299, Validation Accuracy: 14.40%
Epoch [9/10], Training Loss: 2.297, Validation Accuracy: 14.19%
Epoch [10/10], Training Loss: 2.296, Validation Accuracy: 13.73%
Epoch [1/10], Training Loss: 2.296, Validation Accuracy: 14.16%
Epoch [2/10], Training Loss: 2.294, Validation Accuracy: 15.16%
Epoch [3/10], Training Loss: 2.291, Validation Accuracy: 16.58%
Epoch [4/10], Training Loss: 2.288, Validation Accuracy: 18.28%
Epoch [5/10], Train

In [ ]:
import re

# Your provided text
log = """
Epoch [1/10], Training Loss: 2.303, Validation Accuracy: 12.83%
Epoch [2/10], Training Loss: 2.301, Validation Accuracy: 13.90%
Epoch [3/10], Training Loss: 2.299, Validation Accuracy: 13.75%
Epoch [4/10], Training Loss: 2.296, Validation Accuracy: 13.71%
Epoch [5/10], Training Loss: 2.294, Validation Accuracy: 13.32%
Epoch [6/10], Training Loss: 2.290, Validation Accuracy: 13.18%
Epoch [7/10], Training Loss: 2.286, Validation Accuracy: 13.15%
Epoch [8/10], Training Loss: 2.280, Validation Accuracy: 14.79%
Epoch [9/10], Training Loss: 2.273, Validation Accuracy: 17.14%
Epoch [10/10], Training Loss: 2.263, Validation Accuracy: 18.26%
Epoch [1/10], Training Loss: 2.255, Validation Accuracy: 20.06%
Epoch [2/10], Training Loss: 2.239, Validation Accuracy: 21.04%
Epoch [3/10], Training Loss: 2.218, Validation Accuracy: 21.98%
Epoch [4/10], Training Loss: 2.189, Validation Accuracy: 23.01%
Epoch [5/10], Training Loss: 2.158, Validation Accuracy: 23.58%
Epoch [6/10], Training Loss: 2.129, Validation Accuracy: 24.17%
Epoch [7/10], Training Loss: 2.106, Validation Accuracy: 24.46%
Epoch [8/10], Training Loss: 2.088, Validation Accuracy: 25.73%
Epoch [9/10], Training Loss: 2.068, Validation Accuracy: 26.00%
Epoch [10/10], Training Loss: 2.051, Validation Accuracy: 26.60%
Epoch [1/10], Training Loss: 2.033, Validation Accuracy: 26.43%
Epoch [2/10], Training Loss: 2.015, Validation Accuracy: 27.12%
Epoch [3/10], Training Loss: 2.002, Validation Accuracy: 28.09%
Epoch [4/10], Training Loss: 1.985, Validation Accuracy: 28.11%
Epoch [5/10], Training Loss: 1.969, Validation Accuracy: 27.71%
Epoch [6/10], Training Loss: 1.957, Validation Accuracy: 28.89%
Epoch [7/10], Training Loss: 1.944, Validation Accuracy: 29.40%
Epoch [8/10], Training Loss: 1.929, Validation Accuracy: 30.10%
Epoch [9/10], Training Loss: 1.917, Validation Accuracy: 29.78%
Epoch [10/10], Training Loss: 1.906, Validation Accuracy: 30.90%
Epoch [1/10], Training Loss: 1.909, Validation Accuracy: 31.06%
Epoch [2/10], Training Loss: 1.895, Validation Accuracy: 32.07%
Epoch [3/10], Training Loss: 1.878, Validation Accuracy: 31.93%
Epoch [4/10], Training Loss: 1.868, Validation Accuracy: 32.71%
Epoch [5/10], Training Loss: 1.847, Validation Accuracy: 33.24%
Epoch [6/10], Training Loss: 1.832, Validation Accuracy: 33.91%
Epoch [7/10], Training Loss: 1.818, Validation Accuracy: 34.23%
Epoch [8/10], Training Loss: 1.807, Validation Accuracy: 33.68%
Epoch [9/10], Training Loss: 1.783, Validation Accuracy: 35.49%
Epoch [10/10], Training Loss: 1.769, Validation Accuracy: 35.47%
Epoch [1/10], Training Loss: 1.758, Validation Accuracy: 35.75%
Epoch [2/10], Training Loss: 1.745, Validation Accuracy: 36.79%
Epoch [3/10], Training Loss: 1.728, Validation Accuracy: 36.79%
Epoch [4/10], Training Loss: 1.709, Validation Accuracy: 37.37%
Epoch [5/10], Training Loss: 1.697, Validation Accuracy: 38.20%
Epoch [6/10], Training Loss: 1.689, Validation Accuracy: 37.31%
Epoch [7/10], Training Loss: 1.675, Validation Accuracy: 38.47%
Epoch [8/10], Training Loss: 1.663, Validation Accuracy: 38.77%
Epoch [9/10], Training Loss: 1.649, Validation Accuracy: 38.81%
Epoch [10/10], Training Loss: 1.643, Validation Accuracy: 39.65%
Epoch [1/10], Training Loss: 1.657, Validation Accuracy: 38.48%
Epoch [2/10], Training Loss: 1.644, Validation Accuracy: 39.89%
Epoch [3/10], Training Loss: 1.625, Validation Accuracy: 39.54%
Epoch [4/10], Training Loss: 1.616, Validation Accuracy: 40.10%
Epoch [5/10], Training Loss: 1.607, Validation Accuracy: 39.66%
Epoch [6/10], Training Loss: 1.597, Validation Accuracy: 41.13%
Epoch [7/10], Training Loss: 1.585, Validation Accuracy: 41.32%
Epoch [8/10], Training Loss: 1.571, Validation Accuracy: 41.44%
Epoch [9/10], Training Loss: 1.569, Validation Accuracy: 41.23%
Epoch [10/10], Training Loss: 1.560, Validation Accuracy: 42.28%
Epoch [1/10], Training Loss: 1.592, Validation Accuracy: 41.58%
Epoch [2/10], Training Loss: 1.573, Validation Accuracy: 42.96%
Epoch [3/10], Training Loss: 1.564, Validation Accuracy: 42.10%
Epoch [4/10], Training Loss: 1.558, Validation Accuracy: 42.86%
Epoch [5/10], Training Loss: 1.542, Validation Accuracy: 42.91%
Epoch [6/10], Training Loss: 1.538, Validation Accuracy: 43.72%
Epoch [7/10], Training Loss: 1.521, Validation Accuracy: 44.11%
Epoch [8/10], Training Loss: 1.515, Validation Accuracy: 43.01%
Epoch [9/10], Training Loss: 1.506, Validation Accuracy: 44.37%
Epoch [10/10], Training Loss: 1.497, Validation Accuracy: 44.10%
Epoch [1/10], Training Loss: 1.523, Validation Accuracy: 45.51%
Epoch [2/10], Training Loss: 1.499, Validation Accuracy: 45.89%
Epoch [3/10], Training Loss: 1.497, Validation Accuracy: 45.66%
Epoch [4/10], Training Loss: 1.489, Validation Accuracy: 45.72%
Epoch [5/10], Training Loss: 1.473, Validation Accuracy: 45.72%
Epoch [6/10], Training Loss: 1.476, Validation Accuracy: 46.09%
Epoch [7/10], Training Loss: 1.458, Validation Accuracy: 46.52%
Epoch [8/10], Training Loss: 1.448, Validation Accuracy: 45.32%
Epoch [9/10], Training Loss: 1.440, Validation Accuracy: 46.49%
Epoch [10/10], Training Loss: 1.434, Validation Accuracy: 46.78%
Epoch [1/10], Training Loss: 1.487, Validation Accuracy: 47.17%
Epoch [2/10], Training Loss: 1.468, Validation Accuracy: 46.54%
Epoch [3/10], Training Loss: 1.464, Validation Accuracy: 46.79%
Epoch [4/10], Training Loss: 1.455, Validation Accuracy: 46.64%
Epoch [5/10], Training Loss: 1.445, Validation Accuracy: 47.37%
Epoch [6/10], Training Loss: 1.431, Validation Accuracy: 47.88%
Epoch [7/10], Training Loss: 1.434, Validation Accuracy: 47.05%
Epoch [8/10], Training Loss: 1.419, Validation Accuracy: 46.00%
Epoch [9/10], Training Loss: 1.411, Validation Accuracy: 48.17%
Epoch [10/10], Training Loss: 1.408, Validation Accuracy: 47.68%
Epoch [1/10], Training Loss: 1.418, Validation Accuracy: 48.28%
Epoch [2/10], Training Loss: 1.397, Validation Accuracy: 48.63%
Epoch [3/10], Training Loss: 1.385, Validation Accuracy: 48.63%
Epoch [4/10], Training Loss: 1.388, Validation Accuracy: 49.21%
Epoch [5/10], Training Loss: 1.371, Validation Accuracy: 48.54%
Epoch [6/10], Training Loss: 1.361, Validation Accuracy: 49.03%
Epoch [7/10], Training Loss: 1.353, Validation Accuracy: 49.31%
Epoch [8/10], Training Loss: 1.348, Validation Accuracy: 48.64%
Epoch [9/10], Training Loss: 1.336, Validation Accuracy: 49.59%
Epoch [10/10], Training Loss: 1.332, Validation Accuracy: 48.83%
Epoch [1/10], Training Loss: 1.389, Validation Accuracy: 49.60%
Epoch [2/10], Training Loss: 1.379, Validation Accuracy: 49.36%
Epoch [3/10], Training Loss: 1.364, Validation Accuracy: 49.51%
Epoch [4/10], Training Loss: 1.346, Validation Accuracy: 50.27%
Epoch [5/10], Training Loss: 1.339, Validation Accuracy: 49.17%
Epoch [6/10], Training Loss: 1.334, Validation Accuracy: 50.20%
Epoch [7/10], Training Loss: 1.323, Validation Accuracy: 50.70%
Epoch [8/10], Training Loss: 1.314, Validation Accuracy: 50.83%
Epoch [9/10], Training Loss: 1.317, Validation Accuracy: 51.04%
Epoch [10/10], Training Loss: 1.295, Validation Accuracy: 50.90%
Epoch [1/10], Training Loss: 1.353, Validation Accuracy: 51.12%
Epoch [2/10], Training Loss: 1.334, Validation Accuracy: 50.66%
Epoch [3/10], Training Loss: 1.323, Validation Accuracy: 50.79%
Epoch [4/10], Training Loss: 1.305, Validation Accuracy: 50.91%
Epoch [5/10], Training Loss: 1.309, Validation Accuracy: 51.36%
Epoch [6/10], Training Loss: 1.290, Validation Accuracy: 51.19%
Epoch [7/10], Training Loss: 1.285, Validation Accuracy: 51.97%
Epoch [8/10], Training Loss: 1.264, Validation Accuracy: 51.98%
Epoch [9/10], Training Loss: 1.258, Validation Accuracy: 51.67%
Epoch [10/10], Training Loss: 1.245, Validation Accuracy: 52.30%
Epoch [1/10], Training Loss: 1.306, Validation Accuracy: 52.68%
Epoch [2/10], Training Loss: 1.282, Validation Accuracy: 52.61%
Epoch [3/10], Training Loss: 1.273, Validation Accuracy: 51.60%
Epoch [4/10], Training Loss: 1.261, Validation Accuracy: 52.73%
Epoch [5/10], Training Loss: 1.252, Validation Accuracy: 52.83%
Epoch [6/10], Training Loss: 1.237, Validation Accuracy: 52.90%
Epoch [7/10], Training Loss: 1.230, Validation Accuracy: 53.29%
Epoch [8/10], Training Loss: 1.217, Validation Accuracy: 53.07%
Epoch [9/10], Training Loss: 1.207, Validation Accuracy: 53.25%
Epoch [10/10], Training Loss: 1.197, Validation Accuracy: 53.47%
Epoch [1/10], Training Loss: 1.303, Validation Accuracy: 53.27%
Epoch [2/10], Training Loss: 1.279, Validation Accuracy: 53.21%
Epoch [3/10], Training Loss: 1.260, Validation Accuracy: 53.53%
Epoch [4/10], Training Loss: 1.261, Validation Accuracy: 53.14%
Epoch [5/10], Training Loss: 1.235, Validation Accuracy: 54.40%
Epoch [6/10], Training Loss: 1.228, Validation Accuracy: 52.33%
Epoch [7/10], Training Loss: 1.221, Validation Accuracy: 54.25%
Epoch [8/10], Training Loss: 1.211, Validation Accuracy: 54.28%
Epoch [9/10], Training Loss: 1.217, Validation Accuracy: 54.35%
Epoch [10/10], Training Loss: 1.187, Validation Accuracy: 54.45%
Epoch [1/10], Training Loss: 1.229, Validation Accuracy: 54.52%
Epoch [2/10], Training Loss: 1.203, Validation Accuracy: 53.38%
Epoch [3/10], Training Loss: 1.196, Validation Accuracy: 54.87%
Epoch [4/10], Training Loss: 1.183, Validation Accuracy: 54.41%
Epoch [5/10], Training Loss: 1.170, Validation Accuracy: 54.75%
Epoch [6/10], Training Loss: 1.157, Validation Accuracy: 54.81%
Epoch [7/10], Training Loss: 1.152, Validation Accuracy: 54.75%
Epoch [8/10], Training Loss: 1.146, Validation Accuracy: 54.90%
Epoch [9/10], Training Loss: 1.127, Validation Accuracy: 55.68%
Epoch [10/10], Training Loss: 1.112, Validation Accuracy: 54.96%
Epoch [1/10], Training Loss: 1.226, Validation Accuracy: 54.87%
Epoch [2/10], Training Loss: 1.205, Validation Accuracy: 56.23%
Epoch [3/10], Training Loss: 1.180, Validation Accuracy: 56.28%
Epoch [4/10], Training Loss: 1.177, Validation Accuracy: 55.73%
Epoch [5/10], Training Loss: 1.161, Validation Accuracy: 55.92%
Epoch [6/10], Training Loss: 1.150, Validation Accuracy: 55.51%
Epoch [7/10], Training Loss: 1.136, Validation Accuracy: 56.17%
Epoch [8/10], Training Loss: 1.122, Validation Accuracy: 55.57%
Epoch [9/10], Training Loss: 1.133, Validation Accuracy: 55.83%
Epoch [10/10], Training Loss: 1.119, Validation Accuracy: 55.40%
Epoch [1/10], Training Loss: 1.192, Validation Accuracy: 55.89%
Epoch [2/10], Training Loss: 1.165, Validation Accuracy: 56.42%
Epoch [3/10], Training Loss: 1.147, Validation Accuracy: 56.45%
Epoch [4/10], Training Loss: 1.139, Validation Accuracy: 56.49%
Epoch [5/10], Training Loss: 1.127, Validation Accuracy: 56.34%
Epoch [6/10], Training Loss: 1.115, Validation Accuracy: 56.50%
Epoch [7/10], Training Loss: 1.096, Validation Accuracy: 56.75%
Epoch [8/10], Training Loss: 1.094, Validation Accuracy: 56.79%
Epoch [9/10], Training Loss: 1.074, Validation Accuracy: 56.88%
Epoch [10/10], Training Loss: 1.068, Validation Accuracy: 56.70%
Epoch [1/10], Training Loss: 1.167, Validation Accuracy: 56.23%
Epoch [2/10], Training Loss: 1.139, Validation Accuracy: 56.43%
Epoch [3/10], Training Loss: 1.125, Validation Accuracy: 56.47%
Epoch [4/10], Training Loss: 1.110, Validation Accuracy: 56.71%
Epoch [5/10], Training Loss: 1.086, Validation Accuracy: 57.21%
Epoch [6/10], Training Loss: 1.077, Validation Accuracy: 56.91%
Epoch [7/10], Training Loss: 1.071, Validation Accuracy: 57.27%
Epoch [8/10], Training Loss: 1.056, Validation Accuracy: 57.24%
Epoch [9/10], Training Loss: 1.044, Validation Accuracy: 57.29%
Epoch [10/10], Training Loss: 1.034, Validation Accuracy: 57.50%
Epoch [1/10], Training Loss: 1.173, Validation Accuracy: 57.58%
Epoch [2/10], Training Loss: 1.142, Validation Accuracy: 57.52%
Epoch [3/10], Training Loss: 1.129, Validation Accuracy: 57.49%
Epoch [4/10], Training Loss: 1.106, Validation Accuracy: 56.93%
Epoch [5/10], Training Loss: 1.097, Validation Accuracy: 57.93%
Epoch [6/10], Training Loss: 1.082, Validation Accuracy: 57.37%
Epoch [7/10], Training Loss: 1.064, Validation Accuracy: 58.06%
Epoch [8/10], Training Loss: 1.061, Validation Accuracy: 57.28%
Epoch [9/10], Training Loss: 1.051, Validation Accuracy: 57.57%
Epoch [10/10], Training Loss: 1.037, Validation Accuracy: 58.07%
Epoch [1/10], Training Loss: 1.112, Validation Accuracy: 58.75%
Epoch [2/10], Training Loss: 1.077, Validation Accuracy: 58.48%
Epoch [3/10], Training Loss: 1.068, Validation Accuracy: 58.31%
Epoch [4/10], Training Loss: 1.040, Validation Accuracy: 58.31%
Epoch [5/10], Training Loss: 1.031, Validation Accuracy: 58.35%
Epoch [6/10], Training Loss: 1.017, Validation Accuracy: 58.96%
Epoch [7/10], Training Loss: 1.007, Validation Accuracy: 58.32%
Epoch [8/10], Training Loss: 0.996, Validation Accuracy: 57.58%
Epoch [9/10], Training Loss: 0.991, Validation Accuracy: 57.30%
Epoch [10/10], Training Loss: 0.970, Validation Accuracy: 58.13%
Epoch [1/10], Training Loss: 1.117, Validation Accuracy: 59.08%
Epoch [2/10], Training Loss: 1.086, Validation Accuracy: 58.56%
Epoch [3/10], Training Loss: 1.064, Validation Accuracy: 58.90%
Epoch [4/10], Training Loss: 1.045, Validation Accuracy: 58.67%
Epoch [5/10], Training Loss: 1.031, Validation Accuracy: 58.49%
Epoch [6/10], Training Loss: 1.025, Validation Accuracy: 58.01%
Epoch [7/10], Training Loss: 1.011, Validation Accuracy: 58.17%
Epoch [8/10], Training Loss: 1.001, Validation Accuracy: 58.64%
Epoch [9/10], Training Loss: 0.989, Validation Accuracy: 58.74%
Epoch [10/10], Training Loss: 0.972, Validation Accuracy: 58.39%
Epoch [1/10], Training Loss: 1.095, Validation Accuracy: 58.87%
Epoch [2/10], Training Loss: 1.065, Validation Accuracy: 58.27%
Epoch [3/10], Training Loss: 1.045, Validation Accuracy: 58.58%
Epoch [4/10], Training Loss: 1.017, Validation Accuracy: 58.95%
Epoch [5/10], Training Loss: 1.004, Validation Accuracy: 58.32%
Epoch [6/10], Training Loss: 0.995, Validation Accuracy: 58.73%
Epoch [7/10], Training Loss: 0.980, Validation Accuracy: 59.02%
Epoch [8/10], Training Loss: 0.966, Validation Accuracy: 59.24%
Epoch [9/10], Training Loss: 0.950, Validation Accuracy: 58.19%
Epoch [10/10], Training Loss: 0.943, Validation Accuracy: 59.16%
Epoch [1/10], Training Loss: 1.070, Validation Accuracy: 59.55%
Epoch [2/10], Training Loss: 1.038, Validation Accuracy: 59.81%
Epoch [3/10], Training Loss: 1.012, Validation Accuracy: 58.97%
Epoch [4/10], Training Loss: 1.001, Validation Accuracy: 58.43%
Epoch [5/10], Training Loss: 0.980, Validation Accuracy: 59.32%
Epoch [6/10], Training Loss: 0.961, Validation Accuracy: 58.98%
Epoch [7/10], Training Loss: 0.957, Validation Accuracy: 59.76%
Epoch [8/10], Training Loss: 0.940, Validation Accuracy: 59.42%
Epoch [9/10], Training Loss: 0.925, Validation Accuracy: 59.14%
Epoch [10/10], Training Loss: 0.914, Validation Accuracy: 58.97%
Epoch [1/10], Training Loss: 1.079, Validation Accuracy: 58.29%
Epoch [2/10], Training Loss: 1.041, Validation Accuracy: 59.35%
Epoch [3/10], Training Loss: 1.015, Validation Accuracy: 58.38%
Epoch [4/10], Training Loss: 1.000, Validation Accuracy: 59.10%
Epoch [5/10], Training Loss: 0.981, Validation Accuracy: 60.11%
Epoch [6/10], Training Loss: 0.959, Validation Accuracy: 59.97%
Epoch [7/10], Training Loss: 0.971, Validation Accuracy: 59.37%
Epoch [8/10], Training Loss: 0.938, Validation Accuracy: 59.12%
Epoch [9/10], Training Loss: 0.929, Validation Accuracy: 59.25%
Epoch [10/10], Training Loss: 0.913, Validation Accuracy: 58.53%
Epoch [1/10], Training Loss: 1.032, Validation Accuracy: 60.08%
Epoch [2/10], Training Loss: 0.989, Validation Accuracy: 60.20%
Epoch [3/10], Training Loss: 0.962, Validation Accuracy: 60.19%
Epoch [4/10], Training Loss: 0.944, Validation Accuracy: 60.49%
Epoch [5/10], Training Loss: 0.917, Validation Accuracy: 60.72%
Epoch [6/10], Training Loss: 0.901, Validation Accuracy: 60.75%
Epoch [7/10], Training Loss: 0.891, Validation Accuracy: 59.86%
Epoch [8/10], Training Loss: 0.883, Validation Accuracy: 60.12%
Epoch [9/10], Training Loss: 0.882, Validation Accuracy: 59.41%
Epoch [10/10], Training Loss: 0.856, Validation Accuracy: 59.80%
Epoch [1/10], Training Loss: 1.037, Validation Accuracy: 60.30%
Epoch [2/10], Training Loss: 0.992, Validation Accuracy: 59.37%
Epoch [3/10], Training Loss: 0.977, Validation Accuracy: 60.04%
Epoch [4/10], Training Loss: 0.940, Validation Accuracy: 60.51%
Epoch [5/10], Training Loss: 0.938, Validation Accuracy: 60.59%
Epoch [6/10], Training Loss: 0.913, Validation Accuracy: 60.50%
Epoch [7/10], Training Loss: 0.892, Validation Accuracy: 60.47%
Epoch [8/10], Training Loss: 0.883, Validation Accuracy: 60.93%
Epoch [9/10], Training Loss: 0.865, Validation Accuracy: 59.89%
Epoch [10/10], Training Loss: 0.856, Validation Accuracy: 60.48%
Epoch [1/10], Training Loss: 1.016, Validation Accuracy: 59.69%
Epoch [2/10], Training Loss: 0.973, Validation Accuracy: 60.16%
Epoch [3/10], Training Loss: 0.943, Validation Accuracy: 59.19%
Epoch [4/10], Training Loss: 0.922, Validation Accuracy: 60.77%
Epoch [5/10], Training Loss: 0.902, Validation Accuracy: 60.83%
Epoch [6/10], Training Loss: 0.898, Validation Accuracy: 60.72%
Epoch [7/10], Training Loss: 0.872, Validation Accuracy: 60.36%
Epoch [8/10], Training Loss: 0.857, Validation Accuracy: 59.89%
Epoch [9/10], Training Loss: 0.843, Validation Accuracy: 59.98%
Epoch [10/10], Training Loss: 0.837, Validation Accuracy: 60.78%
Epoch [1/10], Training Loss: 0.996, Validation Accuracy: 61.04%
Epoch [2/10], Training Loss: 0.951, Validation Accuracy: 60.81%
Epoch [3/10], Training Loss: 0.927, Validation Accuracy: 60.25%
Epoch [4/10], Training Loss: 0.899, Validation Accuracy: 59.90%
Epoch [5/10], Training Loss: 0.876, Validation Accuracy: 60.73%
Epoch [6/10], Training Loss: 0.861, Validation Accuracy: 60.10%
Epoch [7/10], Training Loss: 0.848, Validation Accuracy: 60.60%
Epoch [8/10], Training Loss: 0.834, Validation Accuracy: 60.50%
Epoch [9/10], Training Loss: 0.818, Validation Accuracy: 60.37%
Epoch [10/10], Training Loss: 0.805, Validation Accuracy: 60.26%
Epoch [1/10], Training Loss: 1.013, Validation Accuracy: 60.41%
Epoch [2/10], Training Loss: 0.953, Validation Accuracy: 60.18%
Epoch [3/10], Training Loss: 0.929, Validation Accuracy: 60.56%
Epoch [4/10], Training Loss: 0.913, Validation Accuracy: 60.99%
Epoch [5/10], Training Loss: 0.880, Validation Accuracy: 60.22%
Epoch [6/10], Training Loss: 0.866, Validation Accuracy: 60.36%
Epoch [7/10], Training Loss: 0.852, Validation Accuracy: 60.31%
Epoch [8/10], Training Loss: 0.830, Validation Accuracy: 60.62%
Epoch [9/10], Training Loss: 0.817, Validation Accuracy: 59.99%
Epoch [10/10], Training Loss: 0.797, Validation Accuracy: 60.52%
Epoch [1/10], Training Loss: 0.954, Validation Accuracy: 61.42%
Epoch [2/10], Training Loss: 0.912, Validation Accuracy: 60.86%
Epoch [3/10], Training Loss: 0.874, Validation Accuracy: 61.14%
Epoch [4/10], Training Loss: 0.843, Validation Accuracy: 60.90%
Epoch [5/10], Training Loss: 0.835, Validation Accuracy: 61.34%
Epoch [6/10], Training Loss: 0.809, Validation Accuracy: 61.17%
Epoch [7/10], Training Loss: 0.794, Validation Accuracy: 61.31%
Epoch [8/10], Training Loss: 0.776, Validation Accuracy: 61.38%
Epoch [9/10], Training Loss: 0.766, Validation Accuracy: 60.25%
Epoch [10/10], Training Loss: 0.754, Validation Accuracy: 60.75%
Epoch [1/10], Training Loss: 0.970, Validation Accuracy: 61.25%
Epoch [2/10], Training Loss: 0.903, Validation Accuracy: 61.02%
Epoch [3/10], Training Loss: 0.884, Validation Accuracy: 59.60%
Epoch [4/10], Training Loss: 0.861, Validation Accuracy: 60.75%
Epoch [5/10], Training Loss: 0.833, Validation Accuracy: 61.65%
Epoch [6/10], Training Loss: 0.812, Validation Accuracy: 60.80%
Epoch [7/10], Training Loss: 0.795, Validation Accuracy: 61.03%
Epoch [8/10], Training Loss: 0.778, Validation Accuracy: 60.56%
Epoch [9/10], Training Loss: 0.768, Validation Accuracy: 60.80%
Epoch [10/10], Training Loss: 0.745, Validation Accuracy: 60.57%
Epoch [1/10], Training Loss: 0.954, Validation Accuracy: 59.89%
Epoch [2/10], Training Loss: 0.893, Validation Accuracy: 60.47%
Epoch [3/10], Training Loss: 0.867, Validation Accuracy: 60.83%
Epoch [4/10], Training Loss: 0.841, Validation Accuracy: 60.61%
Epoch [5/10], Training Loss: 0.815, Validation Accuracy: 61.53%
Epoch [6/10], Training Loss: 0.794, Validation Accuracy: 60.49%
Epoch [7/10], Training Loss: 0.784, Validation Accuracy: 61.39%
Epoch [8/10], Training Loss: 0.756, Validation Accuracy: 61.31%
Epoch [9/10], Training Loss: 0.740, Validation Accuracy: 60.87%
Epoch [10/10], Training Loss: 0.730, Validation Accuracy: 60.91%
Epoch [1/10], Training Loss: 0.935, Validation Accuracy: 60.94%
Epoch [2/10], Training Loss: 0.869, Validation Accuracy: 61.39%
Epoch [3/10], Training Loss: 0.850, Validation Accuracy: 61.70%
Epoch [4/10], Training Loss: 0.809, Validation Accuracy: 61.66%
Epoch [5/10], Training Loss: 0.790, Validation Accuracy: 61.23%
Epoch [6/10], Training Loss: 0.764, Validation Accuracy: 60.52%
Epoch [7/10], Training Loss: 0.745, Validation Accuracy: 61.48%
Epoch [8/10], Training Loss: 0.727, Validation Accuracy: 61.02%
Epoch [9/10], Training Loss: 0.714, Validation Accuracy: 61.23%
Epoch [10/10], Training Loss: 0.697, Validation Accuracy: 60.57%
Epoch [1/10], Training Loss: 0.951, Validation Accuracy: 61.22%
Epoch [2/10], Training Loss: 0.879, Validation Accuracy: 61.37%
Epoch [3/10], Training Loss: 0.844, Validation Accuracy: 61.61%
Epoch [4/10], Training Loss: 0.809, Validation Accuracy: 60.42%
Epoch [5/10], Training Loss: 0.788, Validation Accuracy: 61.59%
Epoch [6/10], Training Loss: 0.765, Validation Accuracy: 60.78%
Epoch [7/10], Training Loss: 0.753, Validation Accuracy: 61.23%
Epoch [8/10], Training Loss: 0.738, Validation Accuracy: 61.23%
Epoch [9/10], Training Loss: 0.714, Validation Accuracy: 61.06%
Epoch [10/10], Training Loss: 0.695, Validation Accuracy: 61.02%
Epoch [1/10], Training Loss: 0.907, Validation Accuracy: 61.44%
Epoch [2/10], Training Loss: 0.832, Validation Accuracy: 61.63%
Epoch [3/10], Training Loss: 0.790, Validation Accuracy: 61.93%
Epoch [4/10], Training Loss: 0.764, Validation Accuracy: 61.38%
Epoch [5/10], Training Loss: 0.745, Validation Accuracy: 61.81%
Epoch [6/10], Training Loss: 0.722, Validation Accuracy: 61.75%
Epoch [7/10], Training Loss: 0.701, Validation Accuracy: 61.79%
Epoch [8/10], Training Loss: 0.682, Validation Accuracy: 61.84%
Epoch [9/10], Training Loss: 0.665, Validation Accuracy: 61.51%
Epoch [10/10], Training Loss: 0.646, Validation Accuracy: 61.23%
Epoch [1/10], Training Loss: 0.907, Validation Accuracy: 61.17%
Epoch [2/10], Training Loss: 0.841, Validation Accuracy: 61.27%
Epoch [3/10], Training Loss: 0.795, Validation Accuracy: 60.24%
Epoch [4/10], Training Loss: 0.766, Validation Accuracy: 61.40%
Epoch [5/10], Training Loss: 0.738, Validation Accuracy: 61.60%
Epoch [6/10], Training Loss: 0.715, Validation Accuracy: 61.22%
Epoch [7/10], Training Loss: 0.699, Validation Accuracy: 61.39%
Epoch [8/10], Training Loss: 0.669, Validation Accuracy: 60.72%
Epoch [9/10], Training Loss: 0.659, Validation Accuracy: 61.16%
Epoch [10/10], Training Loss: 0.648, Validation Accuracy: 61.21%
Epoch [1/10], Training Loss: 0.904, Validation Accuracy: 61.11%
Epoch [2/10], Training Loss: 0.831, Validation Accuracy: 61.05%
Epoch [3/10], Training Loss: 0.781, Validation Accuracy: 61.38%
Epoch [4/10], Training Loss: 0.756, Validation Accuracy: 61.30%
Epoch [5/10], Training Loss: 0.730, Validation Accuracy: 61.46%
Epoch [6/10], Training Loss: 0.707, Validation Accuracy: 61.17%
Epoch [7/10], Training Loss: 0.681, Validation Accuracy: 60.98%
Epoch [8/10], Training Loss: 0.670, Validation Accuracy: 61.24%
Epoch [9/10], Training Loss: 0.644, Validation Accuracy: 60.46%
Epoch [10/10], Training Loss: 0.625, Validation Accuracy: 60.96%
Epoch [1/10], Training Loss: 0.890, Validation Accuracy: 61.34%
Epoch [2/10], Training Loss: 0.807, Validation Accuracy: 60.97%
Epoch [3/10], Training Loss: 0.754, Validation Accuracy: 61.84%
Epoch [4/10], Training Loss: 0.731, Validation Accuracy: 61.24%
Epoch [5/10], Training Loss: 0.700, Validation Accuracy: 61.80%
Epoch [6/10], Training Loss: 0.676, Validation Accuracy: 61.47%
Epoch [7/10], Training Loss: 0.657, Validation Accuracy: 61.38%
Epoch [8/10], Training Loss: 0.633, Validation Accuracy: 60.26%
Epoch [9/10], Training Loss: 0.624, Validation Accuracy: 61.08%
Epoch [10/10], Training Loss: 0.606, Validation Accuracy: 60.96%
Epoch [1/10], Training Loss: 0.898, Validation Accuracy: 61.35%
Epoch [2/10], Training Loss: 0.821, Validation Accuracy: 61.31%
Epoch [3/10], Training Loss: 0.781, Validation Accuracy: 61.45%
Epoch [4/10], Training Loss: 0.739, Validation Accuracy: 60.96%
Epoch [5/10], Training Loss: 0.703, Validation Accuracy: 61.90%
Epoch [6/10], Training Loss: 0.691, Validation Accuracy: 61.33%
Epoch [7/10], Training Loss: 0.665, Validation Accuracy: 61.78%
Epoch [8/10], Training Loss: 0.645, Validation Accuracy: 61.09%
Epoch [9/10], Training Loss: 0.626, Validation Accuracy: 61.14%
Epoch [10/10], Training Loss: 0.604, Validation Accuracy: 60.57%
Epoch [1/10], Training Loss: 0.852, Validation Accuracy: 61.85%
Epoch [2/10], Training Loss: 0.757, Validation Accuracy: 61.66%
Epoch [3/10], Training Loss: 0.720, Validation Accuracy: 61.97%
Epoch [4/10], Training Loss: 0.695, Validation Accuracy: 61.22%
Epoch [5/10], Training Loss: 0.660, Validation Accuracy: 61.33%
Epoch [6/10], Training Loss: 0.631, Validation Accuracy: 61.20%
Epoch [7/10], Training Loss: 0.614, Validation Accuracy: 61.67%
Epoch [8/10], Training Loss: 0.586, Validation Accuracy: 61.61%
Epoch [9/10], Training Loss: 0.565, Validation Accuracy: 61.80%
Epoch [10/10], Training Loss: 0.558, Validation Accuracy: 61.71%
Epoch [1/10], Training Loss: 0.854, Validation Accuracy: 60.96%
Epoch [2/10], Training Loss: 0.770, Validation Accuracy: 62.38%
Epoch [3/10], Training Loss: 0.726, Validation Accuracy: 61.84%
Epoch [4/10], Training Loss: 0.688, Validation Accuracy: 62.00%
Epoch [5/10], Training Loss: 0.655, Validation Accuracy: 61.51%
Epoch [6/10], Training Loss: 0.632, Validation Accuracy: 61.17%
Epoch [7/10], Training Loss: 0.613, Validation Accuracy: 61.35%
Epoch [8/10], Training Loss: 0.583, Validation Accuracy: 61.53%
Epoch [9/10], Training Loss: 0.571, Validation Accuracy: 61.34%
Epoch [10/10], Training Loss: 0.547, Validation Accuracy: 61.11%
Epoch [1/10], Training Loss: 0.843, Validation Accuracy: 60.85%
Epoch [2/10], Training Loss: 0.752, Validation Accuracy: 61.34%
Epoch [3/10], Training Loss: 0.709, Validation Accuracy: 61.19%
Epoch [4/10], Training Loss: 0.673, Validation Accuracy: 61.74%
Epoch [5/10], Training Loss: 0.640, Validation Accuracy: 61.43%
Epoch [6/10], Training Loss: 0.617, Validation Accuracy: 61.28%
Epoch [7/10], Training Loss: 0.587, Validation Accuracy: 61.24%
Epoch [8/10], Training Loss: 0.570, Validation Accuracy: 61.96%
Epoch [9/10], Training Loss: 0.548, Validation Accuracy: 60.80%
Epoch [10/10], Training Loss: 0.525, Validation Accuracy: 60.73%
Epoch [1/10], Training Loss: 0.846, Validation Accuracy: 61.52%
Epoch [2/10], Training Loss: 0.744, Validation Accuracy: 61.81%
Epoch [3/10], Training Loss: 0.686, Validation Accuracy: 61.12%
Epoch [4/10], Training Loss: 0.657, Validation Accuracy: 61.17%
Epoch [5/10], Training Loss: 0.618, Validation Accuracy: 62.16%
Epoch [6/10], Training Loss: 0.580, Validation Accuracy: 61.44%
Epoch [7/10], Training Loss: 0.578, Validation Accuracy: 61.45%
Epoch [8/10], Training Loss: 0.550, Validation Accuracy: 60.98%
Epoch [9/10], Training Loss: 0.527, Validation Accuracy: 61.04%
Epoch [10/10], Training Loss: 0.506, Validation Accuracy: 60.98%
Epoch [1/10], Training Loss: 0.865, Validation Accuracy: 60.35%
Epoch [2/10], Training Loss: 0.756, Validation Accuracy: 61.40%
Epoch [3/10], Training Loss: 0.698, Validation Accuracy: 61.11%
Epoch [4/10], Training Loss: 0.656, Validation Accuracy: 62.16%
Epoch [5/10], Training Loss: 0.622, Validation Accuracy: 61.48%
Epoch [6/10], Training Loss: 0.602, Validation Accuracy: 61.32%
Epoch [7/10], Training Loss: 0.567, Validation Accuracy: 61.46%
Epoch [8/10], Training Loss: 0.544, Validation Accuracy: 61.39%
Epoch [9/10], Training Loss: 0.528, Validation Accuracy: 61.07%
Epoch [10/10], Training Loss: 0.514, Validation Accuracy: 61.23%
Epoch [1/10], Training Loss: 0.815, Validation Accuracy: 61.70%
Epoch [2/10], Training Loss: 0.702, Validation Accuracy: 61.94%
Epoch [3/10], Training Loss: 0.656, Validation Accuracy: 61.65%
Epoch [4/10], Training Loss: 0.613, Validation Accuracy: 61.22%
Epoch [5/10], Training Loss: 0.582, Validation Accuracy: 60.32%
Epoch [6/10], Training Loss: 0.551, Validation Accuracy: 61.66%
Epoch [7/10], Training Loss: 0.519, Validation Accuracy: 60.73%
Epoch [8/10], Training Loss: 0.504, Validation Accuracy: 61.12%
Epoch [9/10], Training Loss: 0.489, Validation Accuracy: 61.75%
Epoch [10/10], Training Loss: 0.463, Validation Accuracy: 61.75%
Epoch [1/10], Training Loss: 0.812, Validation Accuracy: 61.10%
Epoch [2/10], Training Loss: 0.703, Validation Accuracy: 60.93%
Epoch [3/10], Training Loss: 0.655, Validation Accuracy: 60.82%
Epoch [4/10], Training Loss: 0.611, Validation Accuracy: 60.87%
Epoch [5/10], Training Loss: 0.581, Validation Accuracy: 61.81%
Epoch [6/10], Training Loss: 0.551, Validation Accuracy: 61.67%
Epoch [7/10], Training Loss: 0.516, Validation Accuracy: 61.21%
Epoch [8/10], Training Loss: 0.497, Validation Accuracy: 61.46%
Epoch [9/10], Training Loss: 0.470, Validation Accuracy: 61.34%
Epoch [10/10], Training Loss: 0.456, Validation Accuracy: 61.27%
Epoch [1/10], Training Loss: 0.813, Validation Accuracy: 61.48%
Epoch [2/10], Training Loss: 0.692, Validation Accuracy: 60.95%
Epoch [3/10], Training Loss: 0.634, Validation Accuracy: 61.09%
Epoch [4/10], Training Loss: 0.585, Validation Accuracy: 61.35%
Epoch [5/10], Training Loss: 0.555, Validation Accuracy: 61.41%
Epoch [6/10], Training Loss: 0.521, Validation Accuracy: 61.13%
Epoch [7/10], Training Loss: 0.502, Validation Accuracy: 61.38%
Epoch [8/10], Training Loss: 0.472, Validation Accuracy: 60.05%
Epoch [9/10], Training Loss: 0.461, Validation Accuracy: 61.21%
Epoch [10/10], Training Loss: 0.433, Validation Accuracy: 61.59%
Epoch [1/10], Training Loss: 0.807, Validation Accuracy: 61.43%
Epoch [2/10], Training Loss: 0.687, Validation Accuracy: 61.22%
Epoch [3/10], Training Loss: 0.609, Validation Accuracy: 61.78%
Epoch [4/10], Training Loss: 0.567, Validation Accuracy: 62.06%
Epoch [5/10], Training Loss: 0.533, Validation Accuracy: 60.65%
Epoch [6/10], Training Loss: 0.498, Validation Accuracy: 60.99%
Epoch [7/10], Training Loss: 0.479, Validation Accuracy: 61.32%
Epoch [8/10], Training Loss: 0.447, Validation Accuracy: 60.97%
Epoch [9/10], Training Loss: 0.441, Validation Accuracy: 60.80%
Epoch [10/10], Training Loss: 0.415, Validation Accuracy: 60.81%
Epoch [1/10], Training Loss: 0.836, Validation Accuracy: 60.77%
Epoch [2/10], Training Loss: 0.699, Validation Accuracy: 61.00%
Epoch [3/10], Training Loss: 0.629, Validation Accuracy: 60.61%
Epoch [4/10], Training Loss: 0.579, Validation Accuracy: 60.94%
Epoch [5/10], Training Loss: 0.538, Validation Accuracy: 60.41%
Epoch [6/10], Training Loss: 0.515, Validation Accuracy: 61.32%
Epoch [7/10], Training Loss: 0.479, Validation Accuracy: 61.26%
Epoch [8/10], Training Loss: 0.453, Validation Accuracy: 60.49%
Epoch [9/10], Training Loss: 0.439, Validation Accuracy: 60.61%
Epoch [10/10], Training Loss: 0.425, Validation Accuracy: 60.87%
Epoch [1/10], Training Loss: 0.789, Validation Accuracy: 60.64%
Epoch [2/10], Training Loss: 0.645, Validation Accuracy: 61.18%
Epoch [3/10], Training Loss: 0.568, Validation Accuracy: 61.99%
Epoch [4/10], Training Loss: 0.523, Validation Accuracy: 61.66%
Epoch [5/10], Training Loss: 0.497, Validation Accuracy: 61.53%
Epoch [6/10], Training Loss: 0.462, Validation Accuracy: 61.01%
Epoch [7/10], Training Loss: 0.439, Validation Accuracy: 61.53%
Epoch [8/10], Training Loss: 0.417, Validation Accuracy: 60.99%
Epoch [9/10], Training Loss: 0.394, Validation Accuracy: 61.43%
Epoch [10/10], Training Loss: 0.375, Validation Accuracy: 61.10%
"""

# Regular expression to find validation accuracies
accuracies = re.findall(r'Validation Accuracy: (\d+\.\d+)%', log)

# Convert accuracies from string to float
accuracies = [float(acc) for acc in accuracies]

# Print accuracies
print("Accuracies:", accuracies)

# Print size of the array
print("Size of array:", len(accuracies))


Accuracies: [12.83, 13.9, 13.75, 13.71, 13.32, 13.18, 13.15, 14.79, 17.14, 18.26, 20.06, 21.04, 21.98, 23.01, 23.58, 24.17, 24.46, 25.73, 26.0, 26.6, 26.43, 27.12, 28.09, 28.11, 27.71, 28.89, 29.4, 30.1, 29.78, 30.9, 31.06, 32.07, 31.93, 32.71, 33.24, 33.91, 34.23, 33.68, 35.49, 35.47, 35.75, 36.79, 36.79, 37.37, 38.2, 37.31, 38.47, 38.77, 38.81, 39.65, 38.48, 39.89, 39.54, 40.1, 39.66, 41.13, 41.32, 41.44, 41.23, 42.28, 41.58, 42.96, 42.1, 42.86, 42.91, 43.72, 44.11, 43.01, 44.37, 44.1, 45.51, 45.89, 45.66, 45.72, 45.72, 46.09, 46.52, 45.32, 46.49, 46.78, 47.17, 46.54, 46.79, 46.64, 47.37, 47.88, 47.05, 46.0, 48.17, 47.68, 48.28, 48.63, 48.63, 49.21, 48.54, 49.03, 49.31, 48.64, 49.59, 48.83, 49.6, 49.36, 49.51, 50.27, 49.17, 50.2, 50.7, 50.83, 51.04, 50.9, 51.12, 50.66, 50.79, 50.91, 51.36, 51.19, 51.97, 51.98, 51.67, 52.3, 52.68, 52.61, 51.6, 52.73, 52.83, 52.9, 53.29, 53.07, 53.25, 53.47, 53.27, 53.21, 53.53, 53.14, 54.4, 52.33, 54.25, 54.28, 54.35, 54.45, 54.52, 53.38, 54.87, 54.41

In [ ]:
import numpy as np
from scipy.stats import truncnorm
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import Subset, DataLoader
from torchvision.datasets import CIFAR10
from typing import Dict, Tuple
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import random

# Define VAE model
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim, z_dim):
        super(VAE, self).__init__()
        self.x_dim = x_dim
        self.h_dim = h_dim
        self.z_dim = z_dim

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),  # 4x4
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(2048, 256),
            nn.ReLU(),
            nn.Linear(256, 2*z_dim)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 2048),
            nn.ReLU(),
            nn.Unflatten(1, (128, 4, 4)),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, stride=2, padding=1),  # 32x32
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = h[:, :self.z_dim], h[:, self.z_dim:]
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


# Define VAE training procedure
def vae_train(vae: VAE, trainloader: DataLoader, epochs: int) -> None:
    optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    for epoch in range(epochs):
        vae.train()
        for i, data in enumerate(trainloader, 0):
            inputs, _ = data
            optimizer.zero_grad()
            recon_x, mu, logvar = vae(inputs)
            loss = vae_loss(recon_x, inputs, mu, logvar)
            loss.backward()
            optimizer.step()

        scheduler.step()


# Define classification model
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Load CIFAR10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

full_dataset = CIFAR10(root="./data", train=True, download=True, transform=transform)
test_set = CIFAR10(root="./data", train=False, download=True, transform=transform)

# Separate dataset by class
class_indices = {i: [] for i in range(10)}  # CIFAR-10 has 10 classes
for idx, (_, label) in enumerate(full_dataset):
    class_indices[label].append(idx)

# Define target count per class, summing to 60,000 with random distribution
class_counts = np.random.multinomial(60000, [0.1] * 10)  # Adjust probabilities if you want specific class biases
print("Random Images per Class:", class_counts)

# Sample indices based on the specified class counts
indices = []
for class_id, count in enumerate(class_counts):
    # Ensure count does not exceed available images
    count = min(count, len(class_indices[class_id]))
    selected_indices = random.sample(class_indices[class_id], count)
    indices.extend(selected_indices)

# Create a custom CIFAR-10 dataset with the sampled indices
custom_dataset = Subset(full_dataset, indices)
# Split dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_set, val_set = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Create training and validation loaders
trainloader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
valloader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=2)
testloader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)



# Define VAE loss function
def vae_loss(recon_x, x, mu, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Define training procedure for classification model
def train(net: nn.Module, trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    best_acc = 0.0

    for epoch in range(epochs):
        # Training loop
        net.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        scheduler.step()

        # Validation loop
        net.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in valloader:
                images, labels = data
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(net.state_dict(), 'best_model.pth')

        print(f"Epoch [{epoch+1}/{epochs}], Training Loss: {running_loss / (i+1):.3f}, Validation Accuracy: {val_acc:.2f}%")

# Define evaluation procedure
def evaluate(net: nn.Module, testloader: DataLoader) -> Tuple[float, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    net.eval()
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total

    # Calculate confusion matrix
    cm = confusion_matrix(all_labels, all_predictions)
    print(f"Confusion Matrix:\n{cm}")

    # Calculate TP, FP, TN, FN for each class
    tp = np.diag(cm)
    fp = cm.sum(axis=0) - tp
    fn = cm.sum(axis=1) - tp
    tn = cm.sum() - (fp + fn + tp)

    # Calculate precision, recall, and F1 score for each class
    precision = precision_score(all_labels, all_predictions, average=None)
    recall = recall_score(all_labels, all_predictions, average=None)
    f1 = f1_score(all_labels, all_predictions, average=None)

    return accuracy, tp, fp, tn, fn, precision, recall, f1

# Initialize clients
def initialize_clients(trainset, transform, batch_size, num_clients):
    clients = {}
    for i in range(num_clients):
        client_trainset = torch.utils.data.Subset(trainset, range(i * len(trainset) // num_clients, (i + 1) * len(trainset) // num_clients))
        client_trainloader = torch.utils.data.DataLoader(client_trainset, batch_size=batch_size, shuffle=True)
        clients[f"client_{i}"] = client_trainloader
    return clients

def get_distribution_info(vae: VAE) -> Dict:
    # Implement the logic to extract distribution information from the VAE
    # This can involve computing statistics, parameters, or any other relevant information
    # that can be used to generate augmented data

    # Example implementation:
    distribution_info = {
        "uniform": {
            "mean": vae.encoder[-1].bias.data.cpu().numpy(),
            "std": torch.exp(0.5 * vae.encoder[-1].weight.data).cpu().numpy()
        }
    }

    return distribution_info

def send_distribution_info(distribution_info: Dict) -> None:
    # Implement the logic to send the distribution information to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the information

    # Example implementation:
    # Send the distribution information to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass

def generate_augmented_data(vae: VAE, distribution_info_uniform: Dict ) -> torch.Tensor:
    # Generate augmented data using both uniform and truncated uniform distributions

    mean = distribution_info_uniform["mean"].mean().item()  # Convert numpy array to float
    std = distribution_info_uniform["std"].mean().item()  # Convert numpy array to float



     # Generate augmented data using Uniform distribution
    augmented_data_uniform = torch.FloatTensor(64, vae.z_dim).uniform_(mean - std, mean + std)


    # Calculate the average of augmented data from both distributions
    augmented_data_average = augmented_data_uniform

    return augmented_data_average

def federated_train(net: nn.Module, vae: VAE, trainloaders: Dict[str, DataLoader], trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    for epoch in range(epochs):
        for client_id, client_trainloader in trainloaders.items():
            # Train VAE on client data
            vae_train(vae, client_trainloader, epochs=10)

            # Share distribution information with global server
            distribution_info = get_distribution_info(vae)
            send_distribution_info(distribution_info)

            # Receive distribution information from other clients
            other_distribution_info = receive_distribution_info()

            # Generate augmented data using received distribution information
            augmented_data = generate_augmented_data(vae, other_distribution_info["uniform"])

            # Train classification model using local, augmented, and validation data
            train(net, client_trainloader, valloader, epochs=10)

            # Send model updates to global server
            send_model_update(client_id, net.state_dict())

# Define logic to receive distribution information from global server
def receive_distribution_info() -> Dict:
    # Receive distribution information logic
    distribution_info = {
        "uniform": {
            "mean": np.zeros(20),  # Adjust the size based on your latent space dimension
            "std": np.ones(20)
        }
    }
    return distribution_info

def send_model_update(client_id: str, model_update: Dict) -> None:
    # Implement the logic to send the model update to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the model update

    # Example implementation:
    # Send the model update to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass

# Define global server procedure
def global_server() -> None:
    net = Net()
    x_dim = 3 * 32 * 32  # CIFAR-10 input size
    h_dim = 400
    z_dim = 20
    vae = VAE(x_dim, h_dim, z_dim)  # Initialize VAE object with required arguments

    # Initialize clients
    num_clients = 5  # Define the number of clients
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=num_clients)

    # Train model using FedDIS
    federated_train(net, vae, clients, trainloader, valloader, epochs=10)

    # Evaluate final model
    test_accuracy, tp, fp, tn, fn, precision, recall, f1 = evaluate(net, testloader)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

    print("True Positives (TP):", tp)
    print("False Positives (FP):", fp)
    print("True Negatives (TN):", tn)
    print("False Negatives (FN):", fn)
    print(":", fn+tn+tp+fp)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

if __name__ == "__main__":
    global_server()

Files already downloaded and verified
Files already downloaded and verified
Random Images per Class: [6091 6023 6006 5961 5847 6033 6188 5968 5976 5907]
Epoch [1/10], Training Loss: 2.303, Validation Accuracy: 10.22%
Epoch [2/10], Training Loss: 2.303, Validation Accuracy: 10.22%
Epoch [3/10], Training Loss: 2.302, Validation Accuracy: 10.22%
Epoch [4/10], Training Loss: 2.302, Validation Accuracy: 10.22%
Epoch [5/10], Training Loss: 2.301, Validation Accuracy: 10.22%
Epoch [6/10], Training Loss: 2.300, Validation Accuracy: 10.27%
Epoch [7/10], Training Loss: 2.299, Validation Accuracy: 10.44%
Epoch [8/10], Training Loss: 2.297, Validation Accuracy: 11.48%
Epoch [9/10], Training Loss: 2.295, Validation Accuracy: 12.79%
Epoch [10/10], Training Loss: 2.293, Validation Accuracy: 16.07%
Epoch [1/10], Training Loss: 2.290, Validation Accuracy: 16.34%
Epoch [2/10], Training Loss: 2.284, Validation Accuracy: 16.02%
Epoch [3/10], Training Loss: 2.274, Validation Accuracy: 15.54%
Epoch [4/10], 

In [ ]:
import re

# Your provided text
log = """
Epoch [1/10], Training Loss: 2.303, Validation Accuracy: 10.22%
Epoch [2/10], Training Loss: 2.303, Validation Accuracy: 10.22%
Epoch [3/10], Training Loss: 2.302, Validation Accuracy: 10.22%
Epoch [4/10], Training Loss: 2.302, Validation Accuracy: 10.22%
Epoch [5/10], Training Loss: 2.301, Validation Accuracy: 10.22%
Epoch [6/10], Training Loss: 2.300, Validation Accuracy: 10.27%
Epoch [7/10], Training Loss: 2.299, Validation Accuracy: 10.44%
Epoch [8/10], Training Loss: 2.297, Validation Accuracy: 11.48%
Epoch [9/10], Training Loss: 2.295, Validation Accuracy: 12.79%
Epoch [10/10], Training Loss: 2.293, Validation Accuracy: 16.07%
Epoch [1/10], Training Loss: 2.290, Validation Accuracy: 16.34%
Epoch [2/10], Training Loss: 2.284, Validation Accuracy: 16.02%
Epoch [3/10], Training Loss: 2.274, Validation Accuracy: 15.54%
Epoch [4/10], Training Loss: 2.261, Validation Accuracy: 15.61%
Epoch [5/10], Training Loss: 2.248, Validation Accuracy: 16.07%
Epoch [6/10], Training Loss: 2.234, Validation Accuracy: 16.72%
Epoch [7/10], Training Loss: 2.220, Validation Accuracy: 18.66%
Epoch [8/10], Training Loss: 2.206, Validation Accuracy: 20.07%
Epoch [9/10], Training Loss: 2.192, Validation Accuracy: 20.51%
Epoch [10/10], Training Loss: 2.179, Validation Accuracy: 21.34%
Epoch [1/10], Training Loss: 2.177, Validation Accuracy: 21.24%
Epoch [2/10], Training Loss: 2.163, Validation Accuracy: 21.46%
Epoch [3/10], Training Loss: 2.150, Validation Accuracy: 23.00%
Epoch [4/10], Training Loss: 2.139, Validation Accuracy: 23.20%
Epoch [5/10], Training Loss: 2.129, Validation Accuracy: 23.93%
Epoch [6/10], Training Loss: 2.119, Validation Accuracy: 24.03%
Epoch [7/10], Training Loss: 2.108, Validation Accuracy: 24.67%
Epoch [8/10], Training Loss: 2.097, Validation Accuracy: 25.00%
Epoch [9/10], Training Loss: 2.085, Validation Accuracy: 25.54%
Epoch [10/10], Training Loss: 2.073, Validation Accuracy: 25.19%
Epoch [1/10], Training Loss: 2.058, Validation Accuracy: 26.08%
Epoch [2/10], Training Loss: 2.045, Validation Accuracy: 26.84%
Epoch [3/10], Training Loss: 2.033, Validation Accuracy: 27.22%
Epoch [4/10], Training Loss: 2.021, Validation Accuracy: 27.54%
Epoch [5/10], Training Loss: 2.009, Validation Accuracy: 27.62%
Epoch [6/10], Training Loss: 1.998, Validation Accuracy: 27.70%
Epoch [7/10], Training Loss: 1.986, Validation Accuracy: 28.27%
Epoch [8/10], Training Loss: 1.975, Validation Accuracy: 28.65%
Epoch [9/10], Training Loss: 1.966, Validation Accuracy: 27.64%
Epoch [10/10], Training Loss: 1.954, Validation Accuracy: 29.48%
Epoch [1/10], Training Loss: 1.941, Validation Accuracy: 28.98%
Epoch [2/10], Training Loss: 1.926, Validation Accuracy: 29.39%
Epoch [3/10], Training Loss: 1.911, Validation Accuracy: 29.50%
Epoch [4/10], Training Loss: 1.898, Validation Accuracy: 30.02%
Epoch [5/10], Training Loss: 1.884, Validation Accuracy: 30.38%
Epoch [6/10], Training Loss: 1.869, Validation Accuracy: 30.97%
Epoch [7/10], Training Loss: 1.854, Validation Accuracy: 31.25%
Epoch [8/10], Training Loss: 1.838, Validation Accuracy: 31.10%
Epoch [9/10], Training Loss: 1.822, Validation Accuracy: 31.92%
Epoch [10/10], Training Loss: 1.806, Validation Accuracy: 32.82%
Epoch [1/10], Training Loss: 1.812, Validation Accuracy: 33.25%
Epoch [2/10], Training Loss: 1.793, Validation Accuracy: 34.31%
Epoch [3/10], Training Loss: 1.774, Validation Accuracy: 34.87%
Epoch [4/10], Training Loss: 1.757, Validation Accuracy: 35.19%
Epoch [5/10], Training Loss: 1.740, Validation Accuracy: 35.66%
Epoch [6/10], Training Loss: 1.723, Validation Accuracy: 36.75%
Epoch [7/10], Training Loss: 1.711, Validation Accuracy: 36.15%
Epoch [8/10], Training Loss: 1.690, Validation Accuracy: 37.58%
Epoch [9/10], Training Loss: 1.679, Validation Accuracy: 38.20%
Epoch [10/10], Training Loss: 1.660, Validation Accuracy: 38.61%
Epoch [1/10], Training Loss: 1.679, Validation Accuracy: 38.25%
Epoch [2/10], Training Loss: 1.666, Validation Accuracy: 38.87%
Epoch [3/10], Training Loss: 1.645, Validation Accuracy: 39.57%
Epoch [4/10], Training Loss: 1.633, Validation Accuracy: 39.27%
Epoch [5/10], Training Loss: 1.616, Validation Accuracy: 39.75%
Epoch [6/10], Training Loss: 1.604, Validation Accuracy: 40.70%
Epoch [7/10], Training Loss: 1.589, Validation Accuracy: 40.68%
Epoch [8/10], Training Loss: 1.582, Validation Accuracy: 41.86%
Epoch [9/10], Training Loss: 1.571, Validation Accuracy: 42.18%
Epoch [10/10], Training Loss: 1.557, Validation Accuracy: 42.33%
Epoch [1/10], Training Loss: 1.605, Validation Accuracy: 42.83%
Epoch [2/10], Training Loss: 1.587, Validation Accuracy: 42.52%
Epoch [3/10], Training Loss: 1.573, Validation Accuracy: 42.18%
Epoch [4/10], Training Loss: 1.564, Validation Accuracy: 43.16%
Epoch [5/10], Training Loss: 1.555, Validation Accuracy: 43.46%
Epoch [6/10], Training Loss: 1.544, Validation Accuracy: 44.19%
Epoch [7/10], Training Loss: 1.530, Validation Accuracy: 43.57%
Epoch [8/10], Training Loss: 1.525, Validation Accuracy: 44.77%
Epoch [9/10], Training Loss: 1.512, Validation Accuracy: 44.80%
Epoch [10/10], Training Loss: 1.503, Validation Accuracy: 45.07%
Epoch [1/10], Training Loss: 1.503, Validation Accuracy: 44.45%
Epoch [2/10], Training Loss: 1.491, Validation Accuracy: 45.18%
Epoch [3/10], Training Loss: 1.472, Validation Accuracy: 44.21%
Epoch [4/10], Training Loss: 1.474, Validation Accuracy: 45.92%
Epoch [5/10], Training Loss: 1.453, Validation Accuracy: 46.38%
Epoch [6/10], Training Loss: 1.443, Validation Accuracy: 46.53%
Epoch [7/10], Training Loss: 1.434, Validation Accuracy: 46.71%
Epoch [8/10], Training Loss: 1.427, Validation Accuracy: 45.55%
Epoch [9/10], Training Loss: 1.425, Validation Accuracy: 46.73%
Epoch [10/10], Training Loss: 1.411, Validation Accuracy: 47.01%
Epoch [1/10], Training Loss: 1.437, Validation Accuracy: 47.37%
Epoch [2/10], Training Loss: 1.420, Validation Accuracy: 48.08%
Epoch [3/10], Training Loss: 1.411, Validation Accuracy: 47.86%
Epoch [4/10], Training Loss: 1.401, Validation Accuracy: 48.37%
Epoch [5/10], Training Loss: 1.396, Validation Accuracy: 47.69%
Epoch [6/10], Training Loss: 1.382, Validation Accuracy: 48.61%
Epoch [7/10], Training Loss: 1.374, Validation Accuracy: 48.93%
Epoch [8/10], Training Loss: 1.365, Validation Accuracy: 49.18%
Epoch [9/10], Training Loss: 1.357, Validation Accuracy: 48.80%
Epoch [10/10], Training Loss: 1.344, Validation Accuracy: 49.24%
Epoch [1/10], Training Loss: 1.401, Validation Accuracy: 49.76%
Epoch [2/10], Training Loss: 1.376, Validation Accuracy: 49.80%
Epoch [3/10], Training Loss: 1.368, Validation Accuracy: 48.89%
Epoch [4/10], Training Loss: 1.362, Validation Accuracy: 48.65%
Epoch [5/10], Training Loss: 1.348, Validation Accuracy: 48.97%
Epoch [6/10], Training Loss: 1.344, Validation Accuracy: 49.77%
Epoch [7/10], Training Loss: 1.339, Validation Accuracy: 49.89%
Epoch [8/10], Training Loss: 1.324, Validation Accuracy: 50.12%
Epoch [9/10], Training Loss: 1.320, Validation Accuracy: 49.70%
Epoch [10/10], Training Loss: 1.314, Validation Accuracy: 49.97%
Epoch [1/10], Training Loss: 1.373, Validation Accuracy: 49.57%
Epoch [2/10], Training Loss: 1.361, Validation Accuracy: 50.33%
Epoch [3/10], Training Loss: 1.347, Validation Accuracy: 50.78%
Epoch [4/10], Training Loss: 1.332, Validation Accuracy: 50.65%
Epoch [5/10], Training Loss: 1.327, Validation Accuracy: 51.66%
Epoch [6/10], Training Loss: 1.318, Validation Accuracy: 50.50%
Epoch [7/10], Training Loss: 1.304, Validation Accuracy: 50.83%
Epoch [8/10], Training Loss: 1.297, Validation Accuracy: 51.72%
Epoch [9/10], Training Loss: 1.292, Validation Accuracy: 51.48%
Epoch [10/10], Training Loss: 1.285, Validation Accuracy: 51.55%
Epoch [1/10], Training Loss: 1.372, Validation Accuracy: 52.04%
Epoch [2/10], Training Loss: 1.357, Validation Accuracy: 50.80%
Epoch [3/10], Training Loss: 1.342, Validation Accuracy: 51.47%
Epoch [4/10], Training Loss: 1.327, Validation Accuracy: 52.32%
Epoch [5/10], Training Loss: 1.316, Validation Accuracy: 51.89%
Epoch [6/10], Training Loss: 1.305, Validation Accuracy: 52.09%
Epoch [7/10], Training Loss: 1.300, Validation Accuracy: 52.64%
Epoch [8/10], Training Loss: 1.293, Validation Accuracy: 51.79%
Epoch [9/10], Training Loss: 1.281, Validation Accuracy: 52.73%
Epoch [10/10], Training Loss: 1.273, Validation Accuracy: 52.31%
Epoch [1/10], Training Loss: 1.307, Validation Accuracy: 52.50%
Epoch [2/10], Training Loss: 1.296, Validation Accuracy: 52.46%
Epoch [3/10], Training Loss: 1.279, Validation Accuracy: 53.36%
Epoch [4/10], Training Loss: 1.269, Validation Accuracy: 53.10%
Epoch [5/10], Training Loss: 1.258, Validation Accuracy: 53.25%
Epoch [6/10], Training Loss: 1.248, Validation Accuracy: 53.31%
Epoch [7/10], Training Loss: 1.237, Validation Accuracy: 53.36%
Epoch [8/10], Training Loss: 1.227, Validation Accuracy: 53.26%
Epoch [9/10], Training Loss: 1.219, Validation Accuracy: 53.60%
Epoch [10/10], Training Loss: 1.209, Validation Accuracy: 53.26%
Epoch [1/10], Training Loss: 1.273, Validation Accuracy: 54.31%
Epoch [2/10], Training Loss: 1.265, Validation Accuracy: 53.59%
Epoch [3/10], Training Loss: 1.238, Validation Accuracy: 53.57%
Epoch [4/10], Training Loss: 1.229, Validation Accuracy: 52.78%
Epoch [5/10], Training Loss: 1.212, Validation Accuracy: 54.09%
Epoch [6/10], Training Loss: 1.198, Validation Accuracy: 54.12%
Epoch [7/10], Training Loss: 1.192, Validation Accuracy: 54.07%
Epoch [8/10], Training Loss: 1.187, Validation Accuracy: 54.17%
Epoch [9/10], Training Loss: 1.178, Validation Accuracy: 53.90%
Epoch [10/10], Training Loss: 1.172, Validation Accuracy: 54.34%
Epoch [1/10], Training Loss: 1.254, Validation Accuracy: 53.63%
Epoch [2/10], Training Loss: 1.235, Validation Accuracy: 53.90%
Epoch [3/10], Training Loss: 1.233, Validation Accuracy: 54.70%
Epoch [4/10], Training Loss: 1.212, Validation Accuracy: 54.55%
Epoch [5/10], Training Loss: 1.198, Validation Accuracy: 54.01%
Epoch [6/10], Training Loss: 1.198, Validation Accuracy: 54.83%
Epoch [7/10], Training Loss: 1.177, Validation Accuracy: 54.63%
Epoch [8/10], Training Loss: 1.172, Validation Accuracy: 53.69%
Epoch [9/10], Training Loss: 1.170, Validation Accuracy: 55.36%
Epoch [10/10], Training Loss: 1.156, Validation Accuracy: 55.04%
Epoch [1/10], Training Loss: 1.241, Validation Accuracy: 54.65%
Epoch [2/10], Training Loss: 1.223, Validation Accuracy: 55.58%
Epoch [3/10], Training Loss: 1.201, Validation Accuracy: 55.52%
Epoch [4/10], Training Loss: 1.190, Validation Accuracy: 55.71%
Epoch [5/10], Training Loss: 1.175, Validation Accuracy: 55.39%
Epoch [6/10], Training Loss: 1.169, Validation Accuracy: 55.39%
Epoch [7/10], Training Loss: 1.156, Validation Accuracy: 55.80%
Epoch [8/10], Training Loss: 1.143, Validation Accuracy: 55.59%
Epoch [9/10], Training Loss: 1.131, Validation Accuracy: 55.73%
Epoch [10/10], Training Loss: 1.125, Validation Accuracy: 55.11%
Epoch [1/10], Training Loss: 1.244, Validation Accuracy: 55.91%
Epoch [2/10], Training Loss: 1.218, Validation Accuracy: 56.35%
Epoch [3/10], Training Loss: 1.204, Validation Accuracy: 55.29%
Epoch [4/10], Training Loss: 1.186, Validation Accuracy: 55.67%
Epoch [5/10], Training Loss: 1.171, Validation Accuracy: 55.44%
Epoch [6/10], Training Loss: 1.158, Validation Accuracy: 56.12%
Epoch [7/10], Training Loss: 1.145, Validation Accuracy: 55.47%
Epoch [8/10], Training Loss: 1.141, Validation Accuracy: 56.12%
Epoch [9/10], Training Loss: 1.130, Validation Accuracy: 55.27%
Epoch [10/10], Training Loss: 1.123, Validation Accuracy: 56.23%
Epoch [1/10], Training Loss: 1.198, Validation Accuracy: 56.08%
Epoch [2/10], Training Loss: 1.174, Validation Accuracy: 56.88%
Epoch [3/10], Training Loss: 1.154, Validation Accuracy: 56.74%
Epoch [4/10], Training Loss: 1.139, Validation Accuracy: 56.61%
Epoch [5/10], Training Loss: 1.129, Validation Accuracy: 56.43%
Epoch [6/10], Training Loss: 1.111, Validation Accuracy: 56.95%
Epoch [7/10], Training Loss: 1.107, Validation Accuracy: 56.38%
Epoch [8/10], Training Loss: 1.086, Validation Accuracy: 57.09%
Epoch [9/10], Training Loss: 1.086, Validation Accuracy: 57.26%
Epoch [10/10], Training Loss: 1.070, Validation Accuracy: 56.72%
Epoch [1/10], Training Loss: 1.170, Validation Accuracy: 57.23%
Epoch [2/10], Training Loss: 1.141, Validation Accuracy: 57.44%
Epoch [3/10], Training Loss: 1.120, Validation Accuracy: 57.23%
Epoch [4/10], Training Loss: 1.107, Validation Accuracy: 56.99%
Epoch [5/10], Training Loss: 1.089, Validation Accuracy: 57.16%
Epoch [6/10], Training Loss: 1.077, Validation Accuracy: 57.10%
Epoch [7/10], Training Loss: 1.067, Validation Accuracy: 57.01%
Epoch [8/10], Training Loss: 1.056, Validation Accuracy: 56.99%
Epoch [9/10], Training Loss: 1.045, Validation Accuracy: 57.78%
Epoch [10/10], Training Loss: 1.031, Validation Accuracy: 57.30%
Epoch [1/10], Training Loss: 1.166, Validation Accuracy: 57.42%
Epoch [2/10], Training Loss: 1.124, Validation Accuracy: 57.84%
Epoch [3/10], Training Loss: 1.108, Validation Accuracy: 57.43%
Epoch [4/10], Training Loss: 1.094, Validation Accuracy: 57.56%
Epoch [5/10], Training Loss: 1.076, Validation Accuracy: 58.04%
Epoch [6/10], Training Loss: 1.064, Validation Accuracy: 58.21%
Epoch [7/10], Training Loss: 1.054, Validation Accuracy: 58.12%
Epoch [8/10], Training Loss: 1.036, Validation Accuracy: 57.86%
Epoch [9/10], Training Loss: 1.032, Validation Accuracy: 57.62%
Epoch [10/10], Training Loss: 1.023, Validation Accuracy: 57.36%
Epoch [1/10], Training Loss: 1.147, Validation Accuracy: 58.14%
Epoch [2/10], Training Loss: 1.111, Validation Accuracy: 57.97%
Epoch [3/10], Training Loss: 1.089, Validation Accuracy: 58.24%
Epoch [4/10], Training Loss: 1.068, Validation Accuracy: 58.15%
Epoch [5/10], Training Loss: 1.062, Validation Accuracy: 58.05%
Epoch [6/10], Training Loss: 1.048, Validation Accuracy: 58.34%
Epoch [7/10], Training Loss: 1.031, Validation Accuracy: 57.45%
Epoch [8/10], Training Loss: 1.028, Validation Accuracy: 58.09%
Epoch [9/10], Training Loss: 1.011, Validation Accuracy: 58.78%
Epoch [10/10], Training Loss: 0.997, Validation Accuracy: 57.85%
Epoch [1/10], Training Loss: 1.161, Validation Accuracy: 57.93%
Epoch [2/10], Training Loss: 1.113, Validation Accuracy: 58.49%
Epoch [3/10], Training Loss: 1.093, Validation Accuracy: 57.52%
Epoch [4/10], Training Loss: 1.074, Validation Accuracy: 57.97%
Epoch [5/10], Training Loss: 1.052, Validation Accuracy: 58.40%
Epoch [6/10], Training Loss: 1.043, Validation Accuracy: 59.01%
Epoch [7/10], Training Loss: 1.027, Validation Accuracy: 58.55%
Epoch [8/10], Training Loss: 1.020, Validation Accuracy: 58.04%
Epoch [9/10], Training Loss: 1.005, Validation Accuracy: 58.28%
Epoch [10/10], Training Loss: 0.999, Validation Accuracy: 58.60%
Epoch [1/10], Training Loss: 1.110, Validation Accuracy: 58.98%
Epoch [2/10], Training Loss: 1.068, Validation Accuracy: 59.28%
Epoch [3/10], Training Loss: 1.056, Validation Accuracy: 58.76%
Epoch [4/10], Training Loss: 1.028, Validation Accuracy: 59.36%
Epoch [5/10], Training Loss: 1.006, Validation Accuracy: 58.62%
Epoch [6/10], Training Loss: 0.998, Validation Accuracy: 58.78%
Epoch [7/10], Training Loss: 0.985, Validation Accuracy: 59.40%
Epoch [8/10], Training Loss: 0.967, Validation Accuracy: 59.32%
Epoch [9/10], Training Loss: 0.953, Validation Accuracy: 58.93%
Epoch [10/10], Training Loss: 0.945, Validation Accuracy: 58.94%
Epoch [1/10], Training Loss: 1.092, Validation Accuracy: 58.97%
Epoch [2/10], Training Loss: 1.041, Validation Accuracy: 59.30%
Epoch [3/10], Training Loss: 1.019, Validation Accuracy: 58.81%
Epoch [4/10], Training Loss: 1.011, Validation Accuracy: 59.17%
Epoch [5/10], Training Loss: 0.988, Validation Accuracy: 59.51%
Epoch [6/10], Training Loss: 0.966, Validation Accuracy: 59.50%
Epoch [7/10], Training Loss: 0.953, Validation Accuracy: 59.45%
Epoch [8/10], Training Loss: 0.935, Validation Accuracy: 59.39%
Epoch [9/10], Training Loss: 0.932, Validation Accuracy: 59.49%
Epoch [10/10], Training Loss: 0.918, Validation Accuracy: 58.92%
Epoch [1/10], Training Loss: 1.082, Validation Accuracy: 59.24%
Epoch [2/10], Training Loss: 1.027, Validation Accuracy: 59.17%
Epoch [3/10], Training Loss: 1.009, Validation Accuracy: 59.61%
Epoch [4/10], Training Loss: 0.983, Validation Accuracy: 59.28%
Epoch [5/10], Training Loss: 0.974, Validation Accuracy: 58.87%
Epoch [6/10], Training Loss: 0.965, Validation Accuracy: 59.42%
Epoch [7/10], Training Loss: 0.938, Validation Accuracy: 59.14%
Epoch [8/10], Training Loss: 0.923, Validation Accuracy: 59.03%
Epoch [9/10], Training Loss: 0.919, Validation Accuracy: 59.28%
Epoch [10/10], Training Loss: 0.907, Validation Accuracy: 59.20%
Epoch [1/10], Training Loss: 1.064, Validation Accuracy: 60.01%
Epoch [2/10], Training Loss: 1.024, Validation Accuracy: 60.28%
Epoch [3/10], Training Loss: 0.994, Validation Accuracy: 60.32%
Epoch [4/10], Training Loss: 0.980, Validation Accuracy: 59.92%
Epoch [5/10], Training Loss: 0.953, Validation Accuracy: 60.24%
Epoch [6/10], Training Loss: 0.938, Validation Accuracy: 60.16%
Epoch [7/10], Training Loss: 0.923, Validation Accuracy: 59.10%
Epoch [8/10], Training Loss: 0.912, Validation Accuracy: 58.91%
Epoch [9/10], Training Loss: 0.893, Validation Accuracy: 58.84%
Epoch [10/10], Training Loss: 0.887, Validation Accuracy: 59.99%
Epoch [1/10], Training Loss: 1.071, Validation Accuracy: 59.95%
Epoch [2/10], Training Loss: 1.023, Validation Accuracy: 60.28%
Epoch [3/10], Training Loss: 0.996, Validation Accuracy: 59.95%
Epoch [4/10], Training Loss: 0.976, Validation Accuracy: 59.97%
Epoch [5/10], Training Loss: 0.961, Validation Accuracy: 60.12%
Epoch [6/10], Training Loss: 0.953, Validation Accuracy: 60.32%
Epoch [7/10], Training Loss: 0.927, Validation Accuracy: 59.70%
Epoch [8/10], Training Loss: 0.910, Validation Accuracy: 60.23%
Epoch [9/10], Training Loss: 0.897, Validation Accuracy: 59.99%
Epoch [10/10], Training Loss: 0.884, Validation Accuracy: 58.65%
Epoch [1/10], Training Loss: 1.035, Validation Accuracy: 60.19%
Epoch [2/10], Training Loss: 0.983, Validation Accuracy: 60.50%
Epoch [3/10], Training Loss: 0.955, Validation Accuracy: 60.30%
Epoch [4/10], Training Loss: 0.926, Validation Accuracy: 61.11%
Epoch [5/10], Training Loss: 0.913, Validation Accuracy: 60.52%
Epoch [6/10], Training Loss: 0.901, Validation Accuracy: 60.11%
Epoch [7/10], Training Loss: 0.879, Validation Accuracy: 60.59%
Epoch [8/10], Training Loss: 0.865, Validation Accuracy: 60.77%
Epoch [9/10], Training Loss: 0.847, Validation Accuracy: 59.72%
Epoch [10/10], Training Loss: 0.842, Validation Accuracy: 60.56%
Epoch [1/10], Training Loss: 1.019, Validation Accuracy: 60.72%
Epoch [2/10], Training Loss: 0.959, Validation Accuracy: 59.96%
Epoch [3/10], Training Loss: 0.937, Validation Accuracy: 60.29%
Epoch [4/10], Training Loss: 0.922, Validation Accuracy: 60.11%
Epoch [5/10], Training Loss: 0.891, Validation Accuracy: 59.82%
Epoch [6/10], Training Loss: 0.879, Validation Accuracy: 60.96%
Epoch [7/10], Training Loss: 0.851, Validation Accuracy: 59.45%
Epoch [8/10], Training Loss: 0.845, Validation Accuracy: 60.61%
Epoch [9/10], Training Loss: 0.819, Validation Accuracy: 60.35%
Epoch [10/10], Training Loss: 0.814, Validation Accuracy: 60.03%
Epoch [1/10], Training Loss: 1.016, Validation Accuracy: 60.41%
Epoch [2/10], Training Loss: 0.951, Validation Accuracy: 60.52%
Epoch [3/10], Training Loss: 0.919, Validation Accuracy: 60.78%
Epoch [4/10], Training Loss: 0.897, Validation Accuracy: 59.77%
Epoch [5/10], Training Loss: 0.872, Validation Accuracy: 60.94%
Epoch [6/10], Training Loss: 0.868, Validation Accuracy: 60.39%
Epoch [7/10], Training Loss: 0.843, Validation Accuracy: 60.73%
Epoch [8/10], Training Loss: 0.825, Validation Accuracy: 60.22%
Epoch [9/10], Training Loss: 0.818, Validation Accuracy: 60.43%
Epoch [10/10], Training Loss: 0.794, Validation Accuracy: 60.61%
Epoch [1/10], Training Loss: 1.013, Validation Accuracy: 60.49%
Epoch [2/10], Training Loss: 0.945, Validation Accuracy: 61.50%
Epoch [3/10], Training Loss: 0.919, Validation Accuracy: 60.64%
Epoch [4/10], Training Loss: 0.885, Validation Accuracy: 60.93%
Epoch [5/10], Training Loss: 0.860, Validation Accuracy: 61.15%
Epoch [6/10], Training Loss: 0.850, Validation Accuracy: 60.79%
Epoch [7/10], Training Loss: 0.831, Validation Accuracy: 60.49%
Epoch [8/10], Training Loss: 0.808, Validation Accuracy: 60.96%
Epoch [9/10], Training Loss: 0.799, Validation Accuracy: 60.60%
Epoch [10/10], Training Loss: 0.785, Validation Accuracy: 60.42%
Epoch [1/10], Training Loss: 1.011, Validation Accuracy: 60.63%
Epoch [2/10], Training Loss: 0.938, Validation Accuracy: 61.38%
Epoch [3/10], Training Loss: 0.918, Validation Accuracy: 61.08%
Epoch [4/10], Training Loss: 0.881, Validation Accuracy: 60.46%
Epoch [5/10], Training Loss: 0.869, Validation Accuracy: 60.21%
Epoch [6/10], Training Loss: 0.847, Validation Accuracy: 60.59%
Epoch [7/10], Training Loss: 0.828, Validation Accuracy: 60.93%
Epoch [8/10], Training Loss: 0.816, Validation Accuracy: 60.90%
Epoch [9/10], Training Loss: 0.800, Validation Accuracy: 60.37%
Epoch [10/10], Training Loss: 0.780, Validation Accuracy: 60.75%
Epoch [1/10], Training Loss: 0.971, Validation Accuracy: 61.08%
Epoch [2/10], Training Loss: 0.905, Validation Accuracy: 61.22%
Epoch [3/10], Training Loss: 0.873, Validation Accuracy: 61.04%
Epoch [4/10], Training Loss: 0.853, Validation Accuracy: 60.78%
Epoch [5/10], Training Loss: 0.830, Validation Accuracy: 61.20%
Epoch [6/10], Training Loss: 0.803, Validation Accuracy: 60.84%
Epoch [7/10], Training Loss: 0.792, Validation Accuracy: 59.53%
Epoch [8/10], Training Loss: 0.776, Validation Accuracy: 61.10%
Epoch [9/10], Training Loss: 0.756, Validation Accuracy: 60.57%
Epoch [10/10], Training Loss: 0.740, Validation Accuracy: 61.06%
Epoch [1/10], Training Loss: 0.964, Validation Accuracy: 60.35%
Epoch [2/10], Training Loss: 0.905, Validation Accuracy: 61.11%
Epoch [3/10], Training Loss: 0.860, Validation Accuracy: 61.29%
Epoch [4/10], Training Loss: 0.829, Validation Accuracy: 60.95%
Epoch [5/10], Training Loss: 0.804, Validation Accuracy: 61.19%
Epoch [6/10], Training Loss: 0.786, Validation Accuracy: 60.87%
Epoch [7/10], Training Loss: 0.765, Validation Accuracy: 60.01%
Epoch [8/10], Training Loss: 0.749, Validation Accuracy: 61.41%
Epoch [9/10], Training Loss: 0.730, Validation Accuracy: 61.28%
Epoch [10/10], Training Loss: 0.715, Validation Accuracy: 61.21%
Epoch [1/10], Training Loss: 0.960, Validation Accuracy: 61.26%
Epoch [2/10], Training Loss: 0.884, Validation Accuracy: 60.28%
Epoch [3/10], Training Loss: 0.841, Validation Accuracy: 60.33%
Epoch [4/10], Training Loss: 0.825, Validation Accuracy: 61.44%
Epoch [5/10], Training Loss: 0.794, Validation Accuracy: 60.29%
Epoch [6/10], Training Loss: 0.786, Validation Accuracy: 61.16%
Epoch [7/10], Training Loss: 0.752, Validation Accuracy: 60.70%
Epoch [8/10], Training Loss: 0.736, Validation Accuracy: 61.28%
Epoch [9/10], Training Loss: 0.725, Validation Accuracy: 60.86%
Epoch [10/10], Training Loss: 0.703, Validation Accuracy: 60.65%
Epoch [1/10], Training Loss: 0.964, Validation Accuracy: 60.52%
Epoch [2/10], Training Loss: 0.880, Validation Accuracy: 61.13%
Epoch [3/10], Training Loss: 0.837, Validation Accuracy: 61.02%
Epoch [4/10], Training Loss: 0.801, Validation Accuracy: 60.74%
Epoch [5/10], Training Loss: 0.776, Validation Accuracy: 61.03%
Epoch [6/10], Training Loss: 0.760, Validation Accuracy: 60.71%
Epoch [7/10], Training Loss: 0.738, Validation Accuracy: 61.42%
Epoch [8/10], Training Loss: 0.725, Validation Accuracy: 61.41%
Epoch [9/10], Training Loss: 0.710, Validation Accuracy: 60.98%
Epoch [10/10], Training Loss: 0.682, Validation Accuracy: 60.33%
Epoch [1/10], Training Loss: 0.955, Validation Accuracy: 61.37%
Epoch [2/10], Training Loss: 0.885, Validation Accuracy: 60.84%
Epoch [3/10], Training Loss: 0.839, Validation Accuracy: 61.90%
Epoch [4/10], Training Loss: 0.802, Validation Accuracy: 60.58%
Epoch [5/10], Training Loss: 0.782, Validation Accuracy: 61.31%
Epoch [6/10], Training Loss: 0.759, Validation Accuracy: 61.21%
Epoch [7/10], Training Loss: 0.733, Validation Accuracy: 60.95%
Epoch [8/10], Training Loss: 0.723, Validation Accuracy: 60.70%
Epoch [9/10], Training Loss: 0.705, Validation Accuracy: 60.68%
Epoch [10/10], Training Loss: 0.694, Validation Accuracy: 60.80%
Epoch [1/10], Training Loss: 0.916, Validation Accuracy: 61.00%
Epoch [2/10], Training Loss: 0.845, Validation Accuracy: 61.24%
Epoch [3/10], Training Loss: 0.806, Validation Accuracy: 61.42%
Epoch [4/10], Training Loss: 0.765, Validation Accuracy: 61.49%
Epoch [5/10], Training Loss: 0.742, Validation Accuracy: 60.33%
Epoch [6/10], Training Loss: 0.722, Validation Accuracy: 61.26%
Epoch [7/10], Training Loss: 0.704, Validation Accuracy: 61.32%
Epoch [8/10], Training Loss: 0.680, Validation Accuracy: 60.75%
Epoch [9/10], Training Loss: 0.664, Validation Accuracy: 60.80%
Epoch [10/10], Training Loss: 0.646, Validation Accuracy: 61.09%
Epoch [1/10], Training Loss: 0.905, Validation Accuracy: 60.94%
Epoch [2/10], Training Loss: 0.821, Validation Accuracy: 61.18%
Epoch [3/10], Training Loss: 0.773, Validation Accuracy: 60.82%
Epoch [4/10], Training Loss: 0.750, Validation Accuracy: 60.96%
Epoch [5/10], Training Loss: 0.725, Validation Accuracy: 60.93%
Epoch [6/10], Training Loss: 0.695, Validation Accuracy: 61.28%
Epoch [7/10], Training Loss: 0.680, Validation Accuracy: 60.41%
Epoch [8/10], Training Loss: 0.660, Validation Accuracy: 61.03%
Epoch [9/10], Training Loss: 0.633, Validation Accuracy: 60.37%
Epoch [10/10], Training Loss: 0.630, Validation Accuracy: 61.11%
Epoch [1/10], Training Loss: 0.901, Validation Accuracy: 60.82%
Epoch [2/10], Training Loss: 0.816, Validation Accuracy: 61.90%
Epoch [3/10], Training Loss: 0.777, Validation Accuracy: 60.88%
Epoch [4/10], Training Loss: 0.745, Validation Accuracy: 60.90%
Epoch [5/10], Training Loss: 0.713, Validation Accuracy: 61.35%
Epoch [6/10], Training Loss: 0.696, Validation Accuracy: 61.29%
Epoch [7/10], Training Loss: 0.676, Validation Accuracy: 61.35%
Epoch [8/10], Training Loss: 0.653, Validation Accuracy: 61.05%
Epoch [9/10], Training Loss: 0.636, Validation Accuracy: 61.32%
Epoch [10/10], Training Loss: 0.609, Validation Accuracy: 60.51%
Epoch [1/10], Training Loss: 0.901, Validation Accuracy: 61.22%
Epoch [2/10], Training Loss: 0.818, Validation Accuracy: 60.65%
Epoch [3/10], Training Loss: 0.764, Validation Accuracy: 61.43%
Epoch [4/10], Training Loss: 0.727, Validation Accuracy: 61.26%
Epoch [5/10], Training Loss: 0.699, Validation Accuracy: 61.89%
Epoch [6/10], Training Loss: 0.681, Validation Accuracy: 61.45%
Epoch [7/10], Training Loss: 0.646, Validation Accuracy: 61.35%
Epoch [8/10], Training Loss: 0.630, Validation Accuracy: 60.99%
Epoch [9/10], Training Loss: 0.618, Validation Accuracy: 60.98%
Epoch [10/10], Training Loss: 0.597, Validation Accuracy: 61.27%
Epoch [1/10], Training Loss: 0.919, Validation Accuracy: 60.88%
Epoch [2/10], Training Loss: 0.818, Validation Accuracy: 60.84%
Epoch [3/10], Training Loss: 0.764, Validation Accuracy: 61.43%
Epoch [4/10], Training Loss: 0.729, Validation Accuracy: 60.86%
Epoch [5/10], Training Loss: 0.702, Validation Accuracy: 61.36%
Epoch [6/10], Training Loss: 0.679, Validation Accuracy: 60.98%
Epoch [7/10], Training Loss: 0.667, Validation Accuracy: 60.80%
Epoch [8/10], Training Loss: 0.633, Validation Accuracy: 60.87%
Epoch [9/10], Training Loss: 0.611, Validation Accuracy: 60.74%
Epoch [10/10], Training Loss: 0.600, Validation Accuracy: 60.46%
Epoch [1/10], Training Loss: 0.869, Validation Accuracy: 60.48%
Epoch [2/10], Training Loss: 0.776, Validation Accuracy: 60.71%
Epoch [3/10], Training Loss: 0.737, Validation Accuracy: 61.46%
Epoch [4/10], Training Loss: 0.694, Validation Accuracy: 61.42%
Epoch [5/10], Training Loss: 0.657, Validation Accuracy: 61.03%
Epoch [6/10], Training Loss: 0.641, Validation Accuracy: 60.06%
Epoch [7/10], Training Loss: 0.615, Validation Accuracy: 60.91%
Epoch [8/10], Training Loss: 0.596, Validation Accuracy: 60.62%
Epoch [9/10], Training Loss: 0.575, Validation Accuracy: 60.86%
Epoch [10/10], Training Loss: 0.558, Validation Accuracy: 60.66%
Epoch [1/10], Training Loss: 0.866, Validation Accuracy: 60.21%
Epoch [2/10], Training Loss: 0.767, Validation Accuracy: 61.07%
Epoch [3/10], Training Loss: 0.709, Validation Accuracy: 60.28%
Epoch [4/10], Training Loss: 0.683, Validation Accuracy: 60.10%
Epoch [5/10], Training Loss: 0.652, Validation Accuracy: 61.27%
Epoch [6/10], Training Loss: 0.625, Validation Accuracy: 60.50%
Epoch [7/10], Training Loss: 0.600, Validation Accuracy: 61.06%
Epoch [8/10], Training Loss: 0.572, Validation Accuracy: 60.99%
Epoch [9/10], Training Loss: 0.555, Validation Accuracy: 61.03%
Epoch [10/10], Training Loss: 0.535, Validation Accuracy: 60.59%
Epoch [1/10], Training Loss: 0.857, Validation Accuracy: 61.03%
Epoch [2/10], Training Loss: 0.760, Validation Accuracy: 60.72%
Epoch [3/10], Training Loss: 0.711, Validation Accuracy: 61.22%
Epoch [4/10], Training Loss: 0.663, Validation Accuracy: 59.98%
Epoch [5/10], Training Loss: 0.649, Validation Accuracy: 60.82%
Epoch [6/10], Training Loss: 0.611, Validation Accuracy: 60.84%
Epoch [7/10], Training Loss: 0.581, Validation Accuracy: 60.79%
Epoch [8/10], Training Loss: 0.564, Validation Accuracy: 60.55%
Epoch [9/10], Training Loss: 0.552, Validation Accuracy: 60.82%
Epoch [10/10], Training Loss: 0.526, Validation Accuracy: 60.48%
Epoch [1/10], Training Loss: 0.881, Validation Accuracy: 60.37%
Epoch [2/10], Training Loss: 0.755, Validation Accuracy: 60.76%
Epoch [3/10], Training Loss: 0.697, Validation Accuracy: 61.37%
Epoch [4/10], Training Loss: 0.657, Validation Accuracy: 60.86%
Epoch [5/10], Training Loss: 0.620, Validation Accuracy: 61.50%
Epoch [6/10], Training Loss: 0.588, Validation Accuracy: 60.81%
Epoch [7/10], Training Loss: 0.576, Validation Accuracy: 61.76%
Epoch [8/10], Training Loss: 0.550, Validation Accuracy: 60.62%
Epoch [9/10], Training Loss: 0.534, Validation Accuracy: 60.90%
Epoch [10/10], Training Loss: 0.518, Validation Accuracy: 61.11%
Epoch [1/10], Training Loss: 0.876, Validation Accuracy: 61.50%
Epoch [2/10], Training Loss: 0.751, Validation Accuracy: 61.59%
Epoch [3/10], Training Loss: 0.701, Validation Accuracy: 60.73%
Epoch [4/10], Training Loss: 0.663, Validation Accuracy: 60.94%
Epoch [5/10], Training Loss: 0.630, Validation Accuracy: 60.23%
Epoch [6/10], Training Loss: 0.592, Validation Accuracy: 60.85%
Epoch [7/10], Training Loss: 0.567, Validation Accuracy: 60.74%
Epoch [8/10], Training Loss: 0.558, Validation Accuracy: 59.98%
Epoch [9/10], Training Loss: 0.526, Validation Accuracy: 59.97%
Epoch [10/10], Training Loss: 0.506, Validation Accuracy: 60.22%
Epoch [1/10], Training Loss: 0.836, Validation Accuracy: 60.60%
Epoch [2/10], Training Loss: 0.725, Validation Accuracy: 60.98%
Epoch [3/10], Training Loss: 0.663, Validation Accuracy: 61.04%
Epoch [4/10], Training Loss: 0.619, Validation Accuracy: 60.43%
Epoch [5/10], Training Loss: 0.589, Validation Accuracy: 61.41%
Epoch [6/10], Training Loss: 0.551, Validation Accuracy: 60.80%
Epoch [7/10], Training Loss: 0.536, Validation Accuracy: 61.42%
Epoch [8/10], Training Loss: 0.509, Validation Accuracy: 61.40%
Epoch [9/10], Training Loss: 0.493, Validation Accuracy: 60.61%
Epoch [10/10], Training Loss: 0.467, Validation Accuracy: 60.28%
Epoch [1/10], Training Loss: 0.827, Validation Accuracy: 60.57%
Epoch [2/10], Training Loss: 0.697, Validation Accuracy: 60.26%
Epoch [3/10], Training Loss: 0.639, Validation Accuracy: 60.11%
Epoch [4/10], Training Loss: 0.592, Validation Accuracy: 60.82%
Epoch [5/10], Training Loss: 0.564, Validation Accuracy: 60.84%
Epoch [6/10], Training Loss: 0.535, Validation Accuracy: 60.83%
Epoch [7/10], Training Loss: 0.511, Validation Accuracy: 59.87%
Epoch [8/10], Training Loss: 0.489, Validation Accuracy: 61.02%
Epoch [9/10], Training Loss: 0.464, Validation Accuracy: 60.50%
Epoch [10/10], Training Loss: 0.450, Validation Accuracy: 59.91%
"""

# Regular expression to find validation accuracies
accuracies = re.findall(r'Validation Accuracy: (\d+\.\d+)%', log)

# Convert accuracies from string to float
accuracies = [float(acc) for acc in accuracies]

# Print accuracies
print("Accuracies:", accuracies)

# Print size of the array
print("Size of array:", len(accuracies))


Accuracies: [10.22, 10.22, 10.22, 10.22, 10.22, 10.27, 10.44, 11.48, 12.79, 16.07, 16.34, 16.02, 15.54, 15.61, 16.07, 16.72, 18.66, 20.07, 20.51, 21.34, 21.24, 21.46, 23.0, 23.2, 23.93, 24.03, 24.67, 25.0, 25.54, 25.19, 26.08, 26.84, 27.22, 27.54, 27.62, 27.7, 28.27, 28.65, 27.64, 29.48, 28.98, 29.39, 29.5, 30.02, 30.38, 30.97, 31.25, 31.1, 31.92, 32.82, 33.25, 34.31, 34.87, 35.19, 35.66, 36.75, 36.15, 37.58, 38.2, 38.61, 38.25, 38.87, 39.57, 39.27, 39.75, 40.7, 40.68, 41.86, 42.18, 42.33, 42.83, 42.52, 42.18, 43.16, 43.46, 44.19, 43.57, 44.77, 44.8, 45.07, 44.45, 45.18, 44.21, 45.92, 46.38, 46.53, 46.71, 45.55, 46.73, 47.01, 47.37, 48.08, 47.86, 48.37, 47.69, 48.61, 48.93, 49.18, 48.8, 49.24, 49.76, 49.8, 48.89, 48.65, 48.97, 49.77, 49.89, 50.12, 49.7, 49.97, 49.57, 50.33, 50.78, 50.65, 51.66, 50.5, 50.83, 51.72, 51.48, 51.55, 52.04, 50.8, 51.47, 52.32, 51.89, 52.09, 52.64, 51.79, 52.73, 52.31, 52.5, 52.46, 53.36, 53.1, 53.25, 53.31, 53.36, 53.26, 53.6, 53.26, 54.31, 53.59, 53.57, 52.

In [ ]:
import numpy as np
from scipy.stats import truncnorm
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import Subset, DataLoader
from torchvision.datasets import CIFAR10
from typing import Dict, Tuple
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import random

# Define VAE model
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim, z_dim):
        super(VAE, self).__init__()
        self.x_dim = x_dim
        self.h_dim = h_dim
        self.z_dim = z_dim

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),  # 4x4
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(2048, 256),
            nn.ReLU(),
            nn.Linear(256, 2*z_dim)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 2048),
            nn.ReLU(),
            nn.Unflatten(1, (128, 4, 4)),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, stride=2, padding=1),  # 32x32
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = h[:, :self.z_dim], h[:, self.z_dim:]
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


# Define VAE training procedure
def vae_train(vae: VAE, trainloader: DataLoader, epochs: int) -> None:
    optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    for epoch in range(epochs):
        vae.train()
        for i, data in enumerate(trainloader, 0):
            inputs, _ = data
            optimizer.zero_grad()
            recon_x, mu, logvar = vae(inputs)
            loss = vae_loss(recon_x, inputs, mu, logvar)
            loss.backward()
            optimizer.step()

        scheduler.step()


# Define classification model
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Load CIFAR10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

full_dataset = CIFAR10(root="./data", train=True, download=True, transform=transform)
test_set = CIFAR10(root="./data", train=False, download=True, transform=transform)

# Separate dataset by class
class_indices = {i: [] for i in range(10)}  # CIFAR-10 has 10 classes
for idx, (_, label) in enumerate(full_dataset):
    class_indices[label].append(idx)

# Define target count per class, summing to 60,000 with random distribution
class_counts = np.random.multinomial(60000, [0.1] * 10)  # Adjust probabilities if you want specific class biases
print("Random Images per Class:", class_counts)

# Sample indices based on the specified class counts
indices = []
for class_id, count in enumerate(class_counts):
    # Ensure count does not exceed available images
    count = min(count, len(class_indices[class_id]))
    selected_indices = random.sample(class_indices[class_id], count)
    indices.extend(selected_indices)

# Create a custom CIFAR-10 dataset with the sampled indices
custom_dataset = Subset(full_dataset, indices)
# Split dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_set, val_set = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Create training and validation loaders
trainloader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
valloader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=2)
testloader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)



# Define VAE loss function
def vae_loss(recon_x, x, mu, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Define training procedure for classification model
def train(net: nn.Module, trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    best_acc = 0.0

    for epoch in range(epochs):
        # Training loop
        net.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        scheduler.step()

        # Validation loop
        net.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in valloader:
                images, labels = data
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(net.state_dict(), 'best_model.pth')

        print(f"Epoch [{epoch+1}/{epochs}], Training Loss: {running_loss / (i+1):.3f}, Validation Accuracy: {val_acc:.2f}%")

# Define evaluation procedure
def evaluate(net: nn.Module, testloader: DataLoader) -> Tuple[float, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    net.eval()
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total

    # Calculate confusion matrix
    cm = confusion_matrix(all_labels, all_predictions)
    print(f"Confusion Matrix:\n{cm}")

    # Calculate TP, FP, TN, FN for each class
    tp = np.diag(cm)
    fp = cm.sum(axis=0) - tp
    fn = cm.sum(axis=1) - tp
    tn = cm.sum() - (fp + fn + tp)

    # Calculate precision, recall, and F1 score for each class
    precision = precision_score(all_labels, all_predictions, average=None)
    recall = recall_score(all_labels, all_predictions, average=None)
    f1 = f1_score(all_labels, all_predictions, average=None)

    return accuracy, tp, fp, tn, fn, precision, recall, f1

# Initialize clients
def initialize_clients(trainset, transform, batch_size, num_clients):
    clients = {}
    for i in range(num_clients):
        client_trainset = torch.utils.data.Subset(trainset, range(i * len(trainset) // num_clients, (i + 1) * len(trainset) // num_clients))
        client_trainloader = torch.utils.data.DataLoader(client_trainset, batch_size=batch_size, shuffle=True)
        clients[f"client_{i}"] = client_trainloader
    return clients

def get_distribution_info(vae: VAE) -> Dict:
    # Implement the logic to extract distribution information from the VAE
    # This can involve computing statistics, parameters, or any other relevant information
    # that can be used to generate augmented data

    # Example implementation:
    distribution_info = {
        "uniform": {
            "mean": vae.encoder[-1].bias.data.cpu().numpy(),
            "std": torch.exp(0.5 * vae.encoder[-1].weight.data).cpu().numpy()
        }
    }

    return distribution_info

def send_distribution_info(distribution_info: Dict) -> None:
    # Implement the logic to send the distribution information to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the information

    # Example implementation:
    # Send the distribution information to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass

def generate_augmented_data(vae: VAE, distribution_info_uniform: Dict ) -> torch.Tensor:
    # Generate augmented data using both uniform and truncated uniform distributions

    mean = distribution_info_uniform["mean"].mean().item()  # Convert numpy array to float
    std = distribution_info_uniform["std"].mean().item()  # Convert numpy array to float



     # Generate augmented data using Uniform distribution
    augmented_data_uniform = torch.FloatTensor(64, vae.z_dim).uniform_(mean - std, mean + std)


    # Calculate the average of augmented data from both distributions
    augmented_data_average = augmented_data_uniform

    return augmented_data_average

def federated_train(net: nn.Module, vae: VAE, trainloaders: Dict[str, DataLoader], trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    for epoch in range(epochs):
        for client_id, client_trainloader in trainloaders.items():
            # Train VAE on client data
            vae_train(vae, client_trainloader, epochs=10)

            # Share distribution information with global server
            distribution_info = get_distribution_info(vae)
            send_distribution_info(distribution_info)

            # Receive distribution information from other clients
            other_distribution_info = receive_distribution_info()

            # Generate augmented data using received distribution information
            augmented_data = generate_augmented_data(vae, other_distribution_info["uniform"])

            # Train classification model using local, augmented, and validation data
            train(net, client_trainloader, valloader, epochs=10)

            # Send model updates to global server
            send_model_update(client_id, net.state_dict())

# Define logic to receive distribution information from global server
def receive_distribution_info() -> Dict:
    # Receive distribution information logic
    distribution_info = {
        "uniform": {
            "mean": np.zeros(20),  # Adjust the size based on your latent space dimension
            "std": np.ones(20)
        }
    }
    return distribution_info

def send_model_update(client_id: str, model_update: Dict) -> None:
    # Implement the logic to send the model update to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the model update

    # Example implementation:
    # Send the model update to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass

# Define global server procedure
def global_server() -> None:
    net = Net()
    x_dim = 3 * 32 * 32  # CIFAR-10 input size
    h_dim = 400
    z_dim = 20
    vae = VAE(x_dim, h_dim, z_dim)  # Initialize VAE object with required arguments

    # Initialize clients
    num_clients = 5  # Define the number of clients
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=num_clients)

    # Train model using FedDIS
    federated_train(net, vae, clients, trainloader, valloader, epochs=10)

    # Evaluate final model
    test_accuracy, tp, fp, tn, fn, precision, recall, f1 = evaluate(net, testloader)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

    print("True Positives (TP):", tp)
    print("False Positives (FP):", fp)
    print("True Negatives (TN):", tn)
    print("False Negatives (FN):", fn)
    print(":", fn+tn+tp+fp)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

if __name__ == "__main__":
    global_server()

100%|██████████| 170M/170M [00:02<00:00, 80.1MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Random Images per Class: [6023 5956 5990 6006 5979 6044 6106 5926 5927 6043]
Epoch [1/10], Training Loss: 2.303, Validation Accuracy: 10.51%
Epoch [2/10], Training Loss: 2.302, Validation Accuracy: 10.51%
Epoch [3/10], Training Loss: 2.301, Validation Accuracy: 10.51%
Epoch [4/10], Training Loss: 2.299, Validation Accuracy: 10.51%
Epoch [5/10], Training Loss: 2.297, Validation Accuracy: 10.51%
Epoch [6/10], Training Loss: 2.295, Validation Accuracy: 10.54%
Epoch [7/10], Training Loss: 2.292, Validation Accuracy: 10.92%
Epoch [8/10], Training Loss: 2.287, Validation Accuracy: 12.57%
Epoch [9/10], Training Loss: 2.281, Validation Accuracy: 15.49%
Epoch [10/10], Training Loss: 2.271, Validation Accuracy: 17.34%
Epoch [1/10], Training Loss: 2.260, Validation Accuracy: 18.80%
Epoch [2/10], Training Loss: 2.239, Validation Accuracy: 20.08%
Epoch [3/10], Training Loss: 2.208, Validation Accuracy: 22.09%
E